# Adding libraries

In [1]:
import pandas as pd
%pip install geopy
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import time
import numpy as np
from IPython.display import display, FileLink
from math import radians, sin, cos, sqrt, atan2
%pip install python-Levenshtein
import Levenshtein

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Cleaning the data frames

## LA_foreclosures_2015

In [2]:
#Importing file
file_name_2015 = 'LA_foreclosures_2015.csv'

LA_foreclosures_2015 = pd.read_csv(file_name_2015)

LA_foreclosures_2015.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8255 entries, 0 to 8254
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   APN                          8255 non-null   int64 
 1   Registered Date              8255 non-null   object
 2   Property Type                8255 non-null   object
 3   Property Address             8255 non-null   object
 4   Property City                8255 non-null   object
 5   Property State               8255 non-null   object
 6   Property Zip                 8255 non-null   int64 
 7   Council District             8255 non-null   int64 
 8   Lender                       8255 non-null   object
 9   Lender Contact               8255 non-null   object
 10  Lender Contact Phone         8253 non-null   object
 11  Property Management          7835 non-null   object
 12  Property Management Contact  7839 non-null   object
 13  Property Management Address  8255

In [3]:
#Turning column names into lower case and replacing white spaces

LA_foreclosures_2015.rename(columns={'Property Mgmt Contact Phone': 'property_management_contact_phone'}, inplace=True)
LA_foreclosures_2015.columns = LA_foreclosures_2015.columns.str.lower()
LA_foreclosures_2015.columns = LA_foreclosures_2015.columns.str.replace(' ', '_')
LA_foreclosures_2015.columns

Index(['apn', 'registered_date', 'property_type', 'property_address',
       'property_city', 'property_state', 'property_zip', 'council_district',
       'lender', 'lender_contact', 'lender_contact_phone',
       'property_management', 'property_management_contact',
       'property_management_address', 'property_management_contact_phone'],
      dtype='object')

In [4]:
#Dropping some columns
LA_foreclosures_2015.drop(columns=['lender_contact_phone', 'property_management_contact', 'property_management_address', 'property_management_contact_phone'], inplace=True)
LA_foreclosures_2015.columns

Index(['apn', 'registered_date', 'property_type', 'property_address',
       'property_city', 'property_state', 'property_zip', 'council_district',
       'lender', 'lender_contact', 'property_management'],
      dtype='object')

In [5]:
#Dropping rows with null values

LA_foreclosures_2015.dropna(inplace=True)
print(LA_foreclosures_2015.shape)
LA_foreclosures_2015.info()

(7835, 11)
<class 'pandas.core.frame.DataFrame'>
Index: 7835 entries, 1 to 8254
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   apn                  7835 non-null   int64 
 1   registered_date      7835 non-null   object
 2   property_type        7835 non-null   object
 3   property_address     7835 non-null   object
 4   property_city        7835 non-null   object
 5   property_state       7835 non-null   object
 6   property_zip         7835 non-null   int64 
 7   council_district     7835 non-null   int64 
 8   lender               7835 non-null   object
 9   lender_contact       7835 non-null   object
 10  property_management  7835 non-null   object
dtypes: int64(3), object(8)
memory usage: 734.5+ KB


In [6]:
#Creating new columns
LA_foreclosures_2015['longitude'] = None
LA_foreclosures_2015['latitude'] = None
LA_foreclosures_2015['foreclosure_year'] = 2015

print(LA_foreclosures_2015.columns)
print(LA_foreclosures_2015.shape)
print(LA_foreclosures_2015['property_address'].head)

Index(['apn', 'registered_date', 'property_type', 'property_address',
       'property_city', 'property_state', 'property_zip', 'council_district',
       'lender', 'lender_contact', 'property_management', 'longitude',
       'latitude', 'foreclosure_year'],
      dtype='object')
(7835, 14)
<bound method NDFrame.head of 1                22400 W MALDEN ST\nLOS ANGELES, CA 93063
2                  8400 N SALE AVE\nLOS ANGELES, CA 93063
3                  8509 N SALE AVE\nLOS ANGELES, CA 93063
4               8432 N RUDNICK AVE\nLOS ANGELES, CA 93063
5             8909 N FARRALONE AVE\nLOS ANGELES, CA 93063
                              ...                        
8250    5511 E BALTIMORE ST\nLOS ANGELES, CA 90042\n(3...
8251    1325 W ROBIDOUX ST\nLOS ANGELES, CA 90744\n(33...
8252    10709 N HADDON AVE\nLOS ANGELES, CA 91331\n(34...
8253    3146 W LARGA AVE\nLOS ANGELES, CA 90039\n(34.1...
8254    3395 1/2 N CAZADOR ST\nLOS ANGELES, CA 90065\n...
Name: property_address, Length: 7835, dt

In [7]:
#I want to see if there is nested longitudinal and latitudinal data in the 'property_address' column

# Define a regular expression pattern to check for the presence of latitude and longitude
coordinate_pattern = r'\(\d+\.\d+, -\d+\.\d+\)'

# Check which rows contain the nested latitude and longitude data
rows_with_coordinates = LA_foreclosures_2015['property_address'].str.contains(coordinate_pattern)

# Get the count of rows with nested coordinates
count_with_coordinates = rows_with_coordinates.sum()

# Display the count
print(f"Number of rows with nested coordinates: {count_with_coordinates} out of {len(LA_foreclosures_2015)} total rows")

Number of rows with nested coordinates: 7505 out of 7835 total rows


In [8]:
# Now I want to extract that nested longitudinal and latitudinal data into my 'longitude' and 'latitude' columns

# Define a regular expression pattern to extract latitude and longitude when present
pattern = r'\(([-\d.]+), ([-\d.]+)\)'

# Use str.extract to create new columns 'latitude' and 'longitude'
coordinates = LA_foreclosures_2015['property_address'].str.extract(pattern)

# Fill NaN values with appropriate defaults (0 in this case, adjust as needed)
coordinates = coordinates.fillna(0)

# Convert 'latitude' and 'longitude' columns to numeric
LA_foreclosures_2015['latitude'] = pd.to_numeric(coordinates[0])
LA_foreclosures_2015['longitude'] = pd.to_numeric(coordinates[1])

# Display the DataFrame
print(LA_foreclosures_2015)

             apn registered_date  property_type  \
1     2004010012      05/01/2015  Single Family   
2     2004011022      28/01/2015  Single Family   
3     2004012013      24/07/2015  Single Family   
4     2004019004      13/04/2015  Single Family   
5     2004023015      08/01/2015  Single Family   
...          ...             ...            ...   
8250  5478009078      06/01/2015  Single Family   
8251  7415016021      06/01/2015  Single Family   
8252  2618009004      24/06/2015  Single Family   
8253  5436029018      08/01/2015   Multi-Family   
8254  5456005028      31/01/2015  Single Family   

                                       property_address property_city  \
1              22400 W MALDEN ST\nLOS ANGELES, CA 93063   LOS ANGELES   
2                8400 N SALE AVE\nLOS ANGELES, CA 93063   LOS ANGELES   
3                8509 N SALE AVE\nLOS ANGELES, CA 93063   LOS ANGELES   
4             8432 N RUDNICK AVE\nLOS ANGELES, CA 93063   LOS ANGELES   
5           8909 N FAR

## LA_foreclosures_2016

In [9]:
#Importing file

file_name_2016 = 'LA_foreclosures_2016.csv'

LA_foreclosures_2016 = pd.read_csv(file_name_2016)

LA_foreclosures_2016.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7638 entries, 0 to 7637
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   APN                        7638 non-null   int64 
 1   RegisteredDate             7638 non-null   object
 2   PropertyType               7638 non-null   object
 3   PropertyAddress            7638 non-null   object
 4   PropertyCity               7635 non-null   object
 5   PropertyState              7638 non-null   object
 6   PropertyZip                7638 non-null   int64 
 7   CouncilDistrict            7638 non-null   int64 
 8   Lender                     7638 non-null   object
 9   LenderContact              7638 non-null   object
 10  LenderContactPhone         7638 non-null   object
 11  PropertyManagement         7134 non-null   object
 12  PropertyManagementContact  7317 non-null   object
 13  PropertyManagementAddress  7317 non-null   object
 14  Property

In [10]:
#Turning column names into lower case and replacing white spaces

LA_foreclosures_2016.rename(columns={'RegisteredDate': 'Registered_Date', 'PropertyType': 'Property_Type', 'PropertyAddress': 'Property_Address', 'PropertyCity': 'Property_City', 'PropertyState': 'Property_State', 'PropertyZip': 'Property_Zip', 'CouncilDistrict': 'Council_District', 'LenderContact': 'Lender_Contact', 'LenderContactPhone': 'Lender_Contact_Phone', 'PropertyManagement': 'Property_Management', 'PropertyManagementContact': 'Property_Management_Contact', 'PropertyManagementAddress': 'Property_Management_Address', 'PropertyMgmtContactPhone': 'Property_Management_Contact_Phone'}, inplace = True)
LA_foreclosures_2016.columns = LA_foreclosures_2016.columns.str.lower()
LA_foreclosures_2016.columns = LA_foreclosures_2016.columns.str.replace(' ', '_')
LA_foreclosures_2016.columns

Index(['apn', 'registered_date', 'property_type', 'property_address',
       'property_city', 'property_state', 'property_zip', 'council_district',
       'lender', 'lender_contact', 'lender_contact_phone',
       'property_management', 'property_management_contact',
       'property_management_address', 'property_management_contact_phone'],
      dtype='object')

In [11]:
#Dropping some columns

LA_foreclosures_2016.drop(columns=['lender_contact_phone', 'property_management_contact', 'property_management_address', 'property_management_contact_phone'], inplace=True)
LA_foreclosures_2016.columns

Index(['apn', 'registered_date', 'property_type', 'property_address',
       'property_city', 'property_state', 'property_zip', 'council_district',
       'lender', 'lender_contact', 'property_management'],
      dtype='object')

In [12]:
#Dropping rows with null values

LA_foreclosures_2016.dropna(inplace=True)
print(LA_foreclosures_2016.shape)
LA_foreclosures_2016.info()

(7132, 11)
<class 'pandas.core.frame.DataFrame'>
Index: 7132 entries, 1 to 7637
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   apn                  7132 non-null   int64 
 1   registered_date      7132 non-null   object
 2   property_type        7132 non-null   object
 3   property_address     7132 non-null   object
 4   property_city        7132 non-null   object
 5   property_state       7132 non-null   object
 6   property_zip         7132 non-null   int64 
 7   council_district     7132 non-null   int64 
 8   lender               7132 non-null   object
 9   lender_contact       7132 non-null   object
 10  property_management  7132 non-null   object
dtypes: int64(3), object(8)
memory usage: 668.6+ KB


In [13]:
#Creating new columns
LA_foreclosures_2016['longitude'] = None
LA_foreclosures_2016['latitude'] = None
LA_foreclosures_2016['foreclosure_year'] = 2016

print(LA_foreclosures_2016.columns)
print(LA_foreclosures_2016.shape)

Index(['apn', 'registered_date', 'property_type', 'property_address',
       'property_city', 'property_state', 'property_zip', 'council_district',
       'lender', 'lender_contact', 'property_management', 'longitude',
       'latitude', 'foreclosure_year'],
      dtype='object')
(7132, 14)


In [14]:
#I want to see if there is nested longitudinal and latitudinal data in the 'property_address' column

# Define a regular expression pattern to check for the presence of latitude and longitude
pattern = r'\(\d+\.\d+, -\d+\.\d+\)'

# Check which rows contain the nested latitude and longitude data
rows_with_coordinates = LA_foreclosures_2016['property_address'].str.contains(pattern)

# Get the count of rows with nested coordinates
count_with_coordinates = rows_with_coordinates.sum()

# Display the count
print(f"Number of rows with nested coordinates: {count_with_coordinates} out of {len(LA_foreclosures_2016)} total rows")

Number of rows with nested coordinates: 6658 out of 7132 total rows


In [15]:
#Extracting the longitude and latitude values from the 'property_on_map' column

# Define a regular expression pattern to extract latitude and longitude when present
pattern = r'\(([-\d.]+), ([-\d.]+)\)'

# Use str.extract to create new columns 'latitude' and 'longitude'
coordinates = LA_foreclosures_2016['property_address'].str.extract(pattern)

# Fill NaN values with appropriate defaults (0 in this case, adjust as needed)
coordinates = coordinates.fillna(0)

# Convert 'latitude' and 'longitude' columns to numeric
LA_foreclosures_2016['latitude'] = pd.to_numeric(coordinates[0])
LA_foreclosures_2016['longitude'] = pd.to_numeric(coordinates[1])

# Display the DataFrame
print(LA_foreclosures_2016)

             apn registered_date  property_type  \
1     5570001008      01/07/2016  Single Family   
2     6119024008      11/07/2016  Single Family   
3     2631026015      12/14/2016  Single Family   
5     5180020022      11/21/2016  Single Family   
6     5043022038      11/30/2016  Single Family   
...          ...             ...            ...   
7633  2414028031      02/09/2016  Single Family   
7634  5037003006      01/04/2016  Single Family   
7635  2348008053      01/06/2016   Multi-Family   
7636  2226021019      03/28/2016  Single Family   
7637  7415012040      01/15/2016   Multi-Family   

                                       property_address    property_city  \
1     2695 N LA CUESTA DR\nLOS ANGELES, CA 90046\n(3...      LOS ANGELES   
2     841 W 148TH ST\nLOS ANGELES, CA 90247\n(33.898...      LOS ANGELES   
3     11826 W SNELLING ST\nLOS ANGELES, CA 91352\n(3...      LOS ANGELES   
5     2912 E PENNSYLVANIA AVE\nLOS ANGELES, CA 90033...      LOS ANGELES   
6     2

## LA_foreclosures_2017

In [16]:
file_name_2017 = 'LA_foreclosures_2017.csv'

LA_foreclosures_2017 = pd.read_csv(file_name_2017)

LA_foreclosures_2017.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1754 entries, 0 to 1753
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   APN                          1754 non-null   int64 
 1   Registered Date              1754 non-null   object
 2   Property Type                1754 non-null   object
 3   Property Address             1754 non-null   object
 4   Property City                1754 non-null   object
 5   Property State               1754 non-null   object
 6   Property Zip                 1754 non-null   int64 
 7   Council District             1754 non-null   int64 
 8   Lender                       1754 non-null   object
 9   Lender Contact               1754 non-null   object
 10  Lender Contact Phone         1754 non-null   object
 11  Property Management          1679 non-null   object
 12  Property Management Contact  1680 non-null   object
 13  Property Management Address  1680

In [17]:
#Turning column names into lower case and replacing white spaces

LA_foreclosures_2017.rename(columns={'Property Mgmt Contact Phone': 'property_management_contact_phone'}, inplace=True)
LA_foreclosures_2017.columns = LA_foreclosures_2017.columns.str.lower()
LA_foreclosures_2017.columns = LA_foreclosures_2017.columns.str.replace(' ', '_')
LA_foreclosures_2017.columns

Index(['apn', 'registered_date', 'property_type', 'property_address',
       'property_city', 'property_state', 'property_zip', 'council_district',
       'lender', 'lender_contact', 'lender_contact_phone',
       'property_management', 'property_management_contact',
       'property_management_address', 'property_management_contact_phone',
       'property_on_map'],
      dtype='object')

In [18]:
#dropping some columns 

LA_foreclosures_2017.drop(columns=['lender_contact_phone', 'property_management_contact', 'property_management_address', 'property_management_contact_phone'], inplace=True)
LA_foreclosures_2017.columns

Index(['apn', 'registered_date', 'property_type', 'property_address',
       'property_city', 'property_state', 'property_zip', 'council_district',
       'lender', 'lender_contact', 'property_management', 'property_on_map'],
      dtype='object')

In [19]:
#Dropping rows with null values

LA_foreclosures_2017.dropna(inplace=True)
print(LA_foreclosures_2017.shape)
LA_foreclosures_2017.info()

(1679, 12)
<class 'pandas.core.frame.DataFrame'>
Index: 1679 entries, 0 to 1753
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   apn                  1679 non-null   int64 
 1   registered_date      1679 non-null   object
 2   property_type        1679 non-null   object
 3   property_address     1679 non-null   object
 4   property_city        1679 non-null   object
 5   property_state       1679 non-null   object
 6   property_zip         1679 non-null   int64 
 7   council_district     1679 non-null   int64 
 8   lender               1679 non-null   object
 9   lender_contact       1679 non-null   object
 10  property_management  1679 non-null   object
 11  property_on_map      1679 non-null   object
dtypes: int64(3), object(9)
memory usage: 170.5+ KB


In [20]:
#Creating new columns
LA_foreclosures_2017['longitude'] = None
LA_foreclosures_2017['latitude'] = None
LA_foreclosures_2017['foreclosure_year'] = 2017

print(LA_foreclosures_2017.columns)
print(LA_foreclosures_2017.shape)

Index(['apn', 'registered_date', 'property_type', 'property_address',
       'property_city', 'property_state', 'property_zip', 'council_district',
       'lender', 'lender_contact', 'property_management', 'property_on_map',
       'longitude', 'latitude', 'foreclosure_year'],
      dtype='object')
(1679, 15)


In [21]:
#Extracting the longitude and latitude values from the 'property_on_map' column

LA_foreclosures_2017_coordinates = LA_foreclosures_2017['property_on_map'].str.extract(r'POINT \(([-\d.]+) ([-\d.]+)\)')
LA_foreclosures_2017['longitude'] = pd.to_numeric(LA_foreclosures_2017_coordinates[0])
LA_foreclosures_2017['latitude'] = pd.to_numeric(LA_foreclosures_2017_coordinates[1])
print(LA_foreclosures_2017)

             apn registered_date  property_type           property_address  \
0     5003028023      01/25/2017  Single Family             1710 W 56TH ST   
10    5512009019      01/04/2017  Single Family           116 S GARDNER ST   
11    6018013002      01/03/2017  Single Family        7406 S LA SALLE AVE   
12    6004007020      06/28/2017   Multi-Family              950 W 59TH DR   
13    5143029070      03/18/2017  Single Family       1100 W WILSHIRE BLVD   
...          ...             ...            ...                        ...   
1748  5032013045      06/20/2017  Single Family          4007 S ABOURNE RD   
1749  2656017016      03/29/2017  Single Family          15507 W TUPPER ST   
1751  7416023018      01/18/2017  Single Family          931 N NEPTUNE AVE   
1752  5094023021      01/10/2017   Multi-Family  3078 W JAMES M. WOOD BLVD   
1753  7445002012      10/24/2017  Single Family           1300 N MEYLER ST   

     property_city property_state  property_zip  council_distri

In [22]:
#drop 'property_on_map' column

LA_foreclosures_2017.drop(columns='property_on_map', inplace=True)


## LA_foreclosures_2018

In [23]:
file_name_2018 = 'LA_foreclosures_2018.csv'

LA_foreclosures_2018 = pd.read_csv(file_name_2018)

LA_foreclosures_2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3112 entries, 0 to 3111
Data columns (total 16 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   APN                                3112 non-null   int64  
 1   Registered Date                    3112 non-null   object 
 2   Property Type                      3112 non-null   object 
 3   Property Address                   3112 non-null   object 
 4   Property City                      3055 non-null   object 
 5   Property State                     3112 non-null   object 
 6   Property Zip                       3055 non-null   float64
 7   Council District                   3112 non-null   int64  
 8   Lender                             3110 non-null   object 
 9   Lender Contact                     3112 non-null   object 
 10  Lender Contact Phone               3112 non-null   object 
 11  Property Management                2999 non-null   objec

In [24]:
#Turning column names into lower case and replacing white spaces

LA_foreclosures_2018.columns = LA_foreclosures_2018.columns.str.lower()
LA_foreclosures_2018.columns = LA_foreclosures_2018.columns.str.replace(' ', '_')
LA_foreclosures_2018.columns

Index(['apn', 'registered_date', 'property_type', 'property_address',
       'property_city', 'property_state', 'property_zip', 'council_district',
       'lender', 'lender_contact', 'lender_contact_phone',
       'property_management', 'property_management_contact',
       'property_management_address', 'property_management_contact_phone',
       'property_on_map'],
      dtype='object')

In [25]:
#dropping some columns 

LA_foreclosures_2018.drop(columns=['lender_contact_phone', 'property_management_contact', 'property_management_address', 'property_management_contact_phone'], inplace=True)
LA_foreclosures_2018.columns

Index(['apn', 'registered_date', 'property_type', 'property_address',
       'property_city', 'property_state', 'property_zip', 'council_district',
       'lender', 'lender_contact', 'property_management', 'property_on_map'],
      dtype='object')

In [26]:
#Dropping rows with null values

LA_foreclosures_2018.dropna(inplace=True)
print(LA_foreclosures_2018.shape)
LA_foreclosures_2018.info()

(2943, 12)
<class 'pandas.core.frame.DataFrame'>
Index: 2943 entries, 3 to 3111
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   apn                  2943 non-null   int64  
 1   registered_date      2943 non-null   object 
 2   property_type        2943 non-null   object 
 3   property_address     2943 non-null   object 
 4   property_city        2943 non-null   object 
 5   property_state       2943 non-null   object 
 6   property_zip         2943 non-null   float64
 7   council_district     2943 non-null   int64  
 8   lender               2943 non-null   object 
 9   lender_contact       2943 non-null   object 
 10  property_management  2943 non-null   object 
 11  property_on_map      2943 non-null   object 
dtypes: float64(1), int64(2), object(9)
memory usage: 298.9+ KB


In [27]:
#Creating new columns
LA_foreclosures_2018['longitude'] = None
LA_foreclosures_2018['latitude'] = None
LA_foreclosures_2018['foreclosure_year'] = 2018

print(LA_foreclosures_2018.columns)
print(LA_foreclosures_2018.shape)

Index(['apn', 'registered_date', 'property_type', 'property_address',
       'property_city', 'property_state', 'property_zip', 'council_district',
       'lender', 'lender_contact', 'property_management', 'property_on_map',
       'longitude', 'latitude', 'foreclosure_year'],
      dtype='object')
(2943, 15)


In [28]:
#Extracting the longitude and latitude values from the 'property_on_map' column

LA_foreclosures_2018_coordinates = LA_foreclosures_2018['property_on_map'].str.extract(r'\((-?\d+\.\d+), (-?\d+\.\d+)\)')
LA_foreclosures_2018['longitude'] = pd.to_numeric(LA_foreclosures_2018_coordinates[1])
LA_foreclosures_2018['latitude'] = pd.to_numeric(LA_foreclosures_2018_coordinates[0])
print(LA_foreclosures_2018)

             apn registered_date  property_type       property_address  \
3     5015020012      01/17/2018   Multi-Family         2071 W 52ND ST   
6     6006027004      11/06/2018  Single Family          500 E 59TH PL   
8     2524021020      01/04/2018  Single Family      13279 W VAUGHN ST   
9     2125007008      06/07/2018  Single Family    6951 N ETIWANDA AVE   
10    2261035019      12/14/2018  Single Family    15468 W MORRISON ST   
...          ...             ...            ...                    ...   
3107  2624006027      01/12/2018  Single Family    9945 N TAMARACK AVE   
3108  4383006022      01/25/2018  Single Family      9847 W PORTOLA DR   
3109  2761013009      01/31/2018  Single Family  9749 N QUAKERTOWN AVE   
3110  2542019010      01/03/2018  Single Family      9651 N SUNLAND PL   
3111  4384018005      08/10/2018  Single Family       9736 W YOAKUM DR   

     property_city property_state  property_zip  council_district  \
3      LOS ANGELES             CA       90

In [29]:
#dropping 'property_on_map'
LA_foreclosures_2018.drop(columns=['property_on_map'], inplace=True)

## LA_foreclosures_2019

In [30]:
file_name_2019 = 'LA_foreclosures_2019.csv'

LA_foreclosures_2019 = pd.read_csv(file_name_2019)

LA_foreclosures_2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3317 entries, 0 to 3316
Data columns (total 18 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   APN                                3317 non-null   int64  
 1   Registered Date                    3317 non-null   object 
 2   Property Type                      3317 non-null   object 
 3   Property Address                   3317 non-null   object 
 4   Property City                      3313 non-null   object 
 5   Property State                     3317 non-null   object 
 6   Property Zip                       3317 non-null   int64  
 7   Council District                   3317 non-null   int64  
 8   Lender                             3311 non-null   object 
 9   Lender Contact                     3317 non-null   object 
 10  Lender Contact Phone               3317 non-null   object 
 11  Property Management                3113 non-null   objec

In [31]:
#Turning column names into lower case and replacing white spaces

LA_foreclosures_2019.columns = LA_foreclosures_2019.columns.str.lower()
LA_foreclosures_2019.columns = LA_foreclosures_2019.columns.str.replace(' ', '_')
LA_foreclosures_2019.columns

Index(['apn', 'registered_date', 'property_type', 'property_address',
       'property_city', 'property_state', 'property_zip', 'council_district',
       'lender', 'lender_contact', 'lender_contact_phone',
       'property_management', 'property_management_contact',
       'property_management_address', 'property_management_contact_phone',
       'longitude', 'latitude', 'property_on_map'],
      dtype='object')

In [32]:
#dropping some columns 

LA_foreclosures_2019.drop(columns=['lender_contact_phone', 'property_management_contact', 'property_management_address', 'property_management_contact_phone', 'property_on_map'], inplace=True)
LA_foreclosures_2019.columns

Index(['apn', 'registered_date', 'property_type', 'property_address',
       'property_city', 'property_state', 'property_zip', 'council_district',
       'lender', 'lender_contact', 'property_management', 'longitude',
       'latitude'],
      dtype='object')

In [33]:
#Dropping rows with null values

LA_foreclosures_2019.dropna(inplace=True)
print(LA_foreclosures_2019.shape)
LA_foreclosures_2019.info()

(3107, 13)
<class 'pandas.core.frame.DataFrame'>
Index: 3107 entries, 0 to 3316
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   apn                  3107 non-null   int64  
 1   registered_date      3107 non-null   object 
 2   property_type        3107 non-null   object 
 3   property_address     3107 non-null   object 
 4   property_city        3107 non-null   object 
 5   property_state       3107 non-null   object 
 6   property_zip         3107 non-null   int64  
 7   council_district     3107 non-null   int64  
 8   lender               3107 non-null   object 
 9   lender_contact       3107 non-null   object 
 10  property_management  3107 non-null   object 
 11  longitude            3107 non-null   float64
 12  latitude             3107 non-null   float64
dtypes: float64(2), int64(3), object(8)
memory usage: 339.8+ KB


In [34]:
#Creating new column
LA_foreclosures_2019['foreclosure_year'] = 2019

print(LA_foreclosures_2019.columns)
print(LA_foreclosures_2019.shape)

Index(['apn', 'registered_date', 'property_type', 'property_address',
       'property_city', 'property_state', 'property_zip', 'council_district',
       'lender', 'lender_contact', 'property_management', 'longitude',
       'latitude', 'foreclosure_year'],
      dtype='object')
(3107, 14)


## LA_foreclosures_2020

In [35]:
file_name_2020 = 'LA_foreclosures_2020.csv'

LA_foreclosures_2020 = pd.read_csv(file_name_2020)

LA_foreclosures_2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3005 entries, 0 to 3004
Data columns (total 18 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   APN                                3005 non-null   int64  
 1   Registered Date                    3005 non-null   object 
 2   Property Type                      3005 non-null   object 
 3   Property Address                   3005 non-null   object 
 4   Property City                      3005 non-null   object 
 5   Property State                     3005 non-null   object 
 6   Property Zip                       3005 non-null   int64  
 7   Council District                   3005 non-null   int64  
 8   Lender                             3005 non-null   object 
 9   Lender Contact                     3005 non-null   object 
 10  Lender Contact Phone               3005 non-null   object 
 11  Property Management                2797 non-null   objec

In [36]:
#Turning column names into lower case and replacing white spaces

LA_foreclosures_2020.rename(columns={'lon': 'longitude', 'lat': 'latitude'}, inplace = True)
LA_foreclosures_2020.columns = LA_foreclosures_2020.columns.str.lower()
LA_foreclosures_2020.columns = LA_foreclosures_2020.columns.str.replace(' ', '_')
LA_foreclosures_2020.columns

Index(['apn', 'registered_date', 'property_type', 'property_address',
       'property_city', 'property_state', 'property_zip', 'council_district',
       'lender', 'lender_contact', 'lender_contact_phone',
       'property_management', 'property_management_contact',
       'property_management_address', 'property_management_contact_phone',
       'longitude', 'latitude', 'property_on_map'],
      dtype='object')

In [37]:
#dropping some columns 

LA_foreclosures_2020.drop(columns=['lender_contact_phone', 'property_management_contact', 'property_management_address', 'property_management_contact_phone', 'property_on_map'], inplace=True)
LA_foreclosures_2020.columns

Index(['apn', 'registered_date', 'property_type', 'property_address',
       'property_city', 'property_state', 'property_zip', 'council_district',
       'lender', 'lender_contact', 'property_management', 'longitude',
       'latitude'],
      dtype='object')

In [38]:
#Dropping rows with null values

LA_foreclosures_2020.dropna(inplace=True)
print(LA_foreclosures_2020.shape)
LA_foreclosures_2020.info()

(2797, 13)
<class 'pandas.core.frame.DataFrame'>
Index: 2797 entries, 1 to 3004
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   apn                  2797 non-null   int64  
 1   registered_date      2797 non-null   object 
 2   property_type        2797 non-null   object 
 3   property_address     2797 non-null   object 
 4   property_city        2797 non-null   object 
 5   property_state       2797 non-null   object 
 6   property_zip         2797 non-null   int64  
 7   council_district     2797 non-null   int64  
 8   lender               2797 non-null   object 
 9   lender_contact       2797 non-null   object 
 10  property_management  2797 non-null   object 
 11  longitude            2797 non-null   float64
 12  latitude             2797 non-null   float64
dtypes: float64(2), int64(3), object(8)
memory usage: 305.9+ KB


In [39]:
#Creating new column
LA_foreclosures_2020['foreclosure_year'] = 2020

print(LA_foreclosures_2020.columns)
print(LA_foreclosures_2020.shape)

Index(['apn', 'registered_date', 'property_type', 'property_address',
       'property_city', 'property_state', 'property_zip', 'council_district',
       'lender', 'lender_contact', 'property_management', 'longitude',
       'latitude', 'foreclosure_year'],
      dtype='object')
(2797, 14)


## LA_foreclosures_2021

In [40]:
file_name_2021 = 'LA_foreclosures_2021.csv'

LA_foreclosures_2021 = pd.read_csv(file_name_2021)

LA_foreclosures_2021.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2134 entries, 0 to 2133
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   APN                        2134 non-null   int64  
 1   Registered Date            2134 non-null   object 
 2   Property Type              2134 non-null   object 
 3   PropertyAddress            2134 non-null   object 
 4   PropertyCity               2134 non-null   object 
 5   PropertyState              2134 non-null   object 
 6   PropertyZip                2134 non-null   int64  
 7   CouncilDistrict            2134 non-null   int64  
 8   Lender                     2134 non-null   object 
 9   LenderContact              2134 non-null   object 
 10  LenderContactPhone         2134 non-null   object 
 11  PropertyManagement         1993 non-null   object 
 12  PropertyManagementContact  1997 non-null   object 
 13  PropertyManagementAddress  1998 non-null   objec

In [41]:
#Turning column names into lower case and replacing white spaces

LA_foreclosures_2021.rename(columns={'PropertyAddress': 'Property_Address', 'PropertyCity': 'Property_City', 'PropertyState': 'Property_State', 'PropertyZip': 'Property_Zip', 'CouncilDistrict': 'Council_District', 'LenderContact': 'Lender_Contact', 'LenderContactPhone': 'Lender_Contact_Phone', 'PropertyManagement': 'Property_Management', 'PropertyManagementContact': 'Property_Management_Contact', 'PropertyManagementAddress': 'Property_Management_Address', 'PropertyMgmtContactPhone': 'Property_Management_Contact_Phone', 'lon': 'longitude', 'lat': 'latitude'}, inplace = True)
LA_foreclosures_2021.columns = LA_foreclosures_2021.columns.str.lower()
LA_foreclosures_2021.columns = LA_foreclosures_2021.columns.str.replace(' ', '_')
LA_foreclosures_2021.columns

Index(['apn', 'registered_date', 'property_type', 'property_address',
       'property_city', 'property_state', 'property_zip', 'council_district',
       'lender', 'lender_contact', 'lender_contact_phone',
       'property_management', 'property_management_contact',
       'property_management_address', 'property_management_contact_phone',
       'longitude', 'latitude', 'property_on_map'],
      dtype='object')

In [42]:
#dropping some columns 

LA_foreclosures_2021.drop(columns=['lender_contact_phone', 'property_management_contact', 'property_management_address', 'property_management_contact_phone', 'property_on_map'], inplace=True)
LA_foreclosures_2021.columns

Index(['apn', 'registered_date', 'property_type', 'property_address',
       'property_city', 'property_state', 'property_zip', 'council_district',
       'lender', 'lender_contact', 'property_management', 'longitude',
       'latitude'],
      dtype='object')

In [43]:
#Dropping rows with null values

LA_foreclosures_2021.dropna(inplace=True)
print(LA_foreclosures_2021.shape)
LA_foreclosures_2021.info()

(1993, 13)
<class 'pandas.core.frame.DataFrame'>
Index: 1993 entries, 1 to 2133
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   apn                  1993 non-null   int64  
 1   registered_date      1993 non-null   object 
 2   property_type        1993 non-null   object 
 3   property_address     1993 non-null   object 
 4   property_city        1993 non-null   object 
 5   property_state       1993 non-null   object 
 6   property_zip         1993 non-null   int64  
 7   council_district     1993 non-null   int64  
 8   lender               1993 non-null   object 
 9   lender_contact       1993 non-null   object 
 10  property_management  1993 non-null   object 
 11  longitude            1993 non-null   float64
 12  latitude             1993 non-null   float64
dtypes: float64(2), int64(3), object(8)
memory usage: 218.0+ KB


In [44]:
#Creating new column
LA_foreclosures_2021['foreclosure_year'] = 2021

print(LA_foreclosures_2021.columns)
print(LA_foreclosures_2021.shape)

Index(['apn', 'registered_date', 'property_type', 'property_address',
       'property_city', 'property_state', 'property_zip', 'council_district',
       'lender', 'lender_contact', 'property_management', 'longitude',
       'latitude', 'foreclosure_year'],
      dtype='object')
(1993, 14)


## LA_foreclosures_2022

In [45]:
file_name_2022 = 'LA_foreclosures_2022.csv'

LA_foreclosures_2022 = pd.read_csv(file_name_2022)

LA_foreclosures_2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2437 entries, 0 to 2436
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   APN                          2437 non-null   int64  
 1   Registered Date              2437 non-null   object 
 2   Property Type                2437 non-null   object 
 3   Property Address             2437 non-null   object 
 4   Property City                2437 non-null   object 
 5   Property State               2437 non-null   object 
 6   Property Zip                 2437 non-null   int64  
 7   Council District             2437 non-null   int64  
 8   Lender                       2437 non-null   object 
 9   Lender Contact               2437 non-null   object 
 10  Lender Contact Phone         2437 non-null   object 
 11  Property Management          2272 non-null   object 
 12  Property Management Contact  2275 non-null   object 
 13  Property Managemen

In [46]:
#Turning column names into lower case and replacing white spaces

LA_foreclosures_2022.rename(columns={'Property Mgmt Contact Phone': 'property_management_contact_phone', 'lon': 'longitude', 'lat': 'latitude'}, inplace=True)
LA_foreclosures_2022.columns = LA_foreclosures_2022.columns.str.lower()
LA_foreclosures_2022.columns = LA_foreclosures_2022.columns.str.replace(' ', '_')
LA_foreclosures_2022.columns

Index(['apn', 'registered_date', 'property_type', 'property_address',
       'property_city', 'property_state', 'property_zip', 'council_district',
       'lender', 'lender_contact', 'lender_contact_phone',
       'property_management', 'property_management_contact',
       'property_management_address', 'property_management_contact_phone',
       'longitude', 'latitude', 'property_on_map'],
      dtype='object')

In [47]:
#dropping some columns 

LA_foreclosures_2022.drop(columns=['lender_contact_phone', 'property_management_contact', 'property_management_address', 'property_management_contact_phone', 'property_on_map'], inplace=True)
LA_foreclosures_2022.columns

Index(['apn', 'registered_date', 'property_type', 'property_address',
       'property_city', 'property_state', 'property_zip', 'council_district',
       'lender', 'lender_contact', 'property_management', 'longitude',
       'latitude'],
      dtype='object')

In [48]:
#Dropping rows with null values

LA_foreclosures_2022.dropna(inplace=True)
print(LA_foreclosures_2022.shape)
LA_foreclosures_2022.info()

(2272, 13)
<class 'pandas.core.frame.DataFrame'>
Index: 2272 entries, 9 to 2436
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   apn                  2272 non-null   int64  
 1   registered_date      2272 non-null   object 
 2   property_type        2272 non-null   object 
 3   property_address     2272 non-null   object 
 4   property_city        2272 non-null   object 
 5   property_state       2272 non-null   object 
 6   property_zip         2272 non-null   int64  
 7   council_district     2272 non-null   int64  
 8   lender               2272 non-null   object 
 9   lender_contact       2272 non-null   object 
 10  property_management  2272 non-null   object 
 11  longitude            2272 non-null   float64
 12  latitude             2272 non-null   float64
dtypes: float64(2), int64(3), object(8)
memory usage: 248.5+ KB


In [49]:
#Creating new column
LA_foreclosures_2022['foreclosure_year'] = 2022

print(LA_foreclosures_2022.columns)
print(LA_foreclosures_2022.shape)

Index(['apn', 'registered_date', 'property_type', 'property_address',
       'property_city', 'property_state', 'property_zip', 'council_district',
       'lender', 'lender_contact', 'property_management', 'longitude',
       'latitude', 'foreclosure_year'],
      dtype='object')
(2272, 14)


# Merging the data frames and more cleaning

In [50]:
#Merging data frames

all_LA_foreclosure_dfs = [LA_foreclosures_2015, LA_foreclosures_2016, LA_foreclosures_2017, LA_foreclosures_2018, LA_foreclosures_2019, LA_foreclosures_2020, LA_foreclosures_2021, LA_foreclosures_2022]

LA_foreclosures_2015_to_2022 = pd.concat(all_LA_foreclosure_dfs, ignore_index=True)

print(LA_foreclosures_2015_to_2022)

              apn         registered_date  property_type  \
0      2004010012              05/01/2015  Single Family   
1      2004011022              28/01/2015  Single Family   
2      2004012013              24/07/2015  Single Family   
3      2004019004              13/04/2015  Single Family   
4      2004023015              08/01/2015  Single Family   
...           ...                     ...            ...   
29753  2116008035  12/21/2022 12:00:00 AM  Single Family   
29754  2733011004  09/27/2022 12:00:00 AM  Single Family   
29755  4119020006  06/07/2022 12:00:00 AM  Single Family   
29756  7351025018  08/05/2022 12:00:00 AM  Single Family   
29757  2620020002  11/03/2022 12:00:00 AM  Single Family   

                                  property_address property_city  \
0         22400 W MALDEN ST\nLOS ANGELES, CA 93063   LOS ANGELES   
1           8400 N SALE AVE\nLOS ANGELES, CA 93063   LOS ANGELES   
2           8509 N SALE AVE\nLOS ANGELES, CA 93063   LOS ANGELES   
3      

In [51]:
#dropping rows with null longitude and latitude values 

# Count of rows where 'latitude' contains "0.00"
count_latitude_zero = LA_foreclosures_2015_to_2022[LA_foreclosures_2015_to_2022['latitude'] == 0.00000].shape[0]

# Count of rows where 'longitude' contains "0.00"
count_longitude_zero = LA_foreclosures_2015_to_2022[LA_foreclosures_2015_to_2022['longitude'] == 0.00000].shape[0]

# Display the counts
print(f"Number of rows with latitude equal to 0.00: {count_latitude_zero}")
print(f"Number of rows with longitude equal to 0.00: {count_longitude_zero}")

#drop the rows with the 0.0000 values
# Create a boolean mask for rows where 'latitude' or 'longitude' contains "0.00"
mask = (LA_foreclosures_2015_to_2022['latitude'] == 0.00) | (LA_foreclosures_2015_to_2022['longitude'] == 0.00)

# Drop the rows based on the boolean mask in place
LA_foreclosures_2015_to_2022.drop(LA_foreclosures_2015_to_2022[mask].index, inplace=True)

# Display the DataFrame after dropping rows
print("DataFrame after dropping rows with latitude or longitude equal to 0.00:")
print(LA_foreclosures_2015_to_2022)

Number of rows with latitude equal to 0.00: 804
Number of rows with longitude equal to 0.00: 804
DataFrame after dropping rows with latitude or longitude equal to 0.00:
              apn         registered_date  property_type  \
311    6083008010              01/12/2015   Multi-Family   
331    2164023003              22/01/2015  Single Family   
332    5143012013              22/04/2015  Single Family   
333    4106029005              15/07/2015  Single Family   
334    2138005033              24/08/2015  Single Family   
...           ...                     ...            ...   
29753  2116008035  12/21/2022 12:00:00 AM  Single Family   
29754  2733011004  09/27/2022 12:00:00 AM  Single Family   
29755  4119020006  06/07/2022 12:00:00 AM  Single Family   
29756  7351025018  08/05/2022 12:00:00 AM  Single Family   
29757  2620020002  11/03/2022 12:00:00 AM  Single Family   

                                        property_address property_city  \
311    346 E 116TH ST\nLOS ANGELES, 

In [52]:
# adding a 'distance_to_city_center' column and caluclating distance


# Function to calculate distance using Haversine formula
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # Earth radius in kilometers

    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Calculate differences in coordinates
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Haversine formula
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    # Distance in kilometers
    distance = R * c

    return distance

# LA city center coordinates
la_city_center = {'latitude': 34.0522, 'longitude': -118.2437}

# Calculate distance for each row in the DataFrame
LA_foreclosures_2015_to_2022['distance_to_la_city_center'] = \
    LA_foreclosures_2015_to_2022.apply(
        lambda row: haversine(row['latitude'], row['longitude'], la_city_center['latitude'], la_city_center['longitude']),
        axis=1
    )

# Display the DataFrame with the new 'distance_to_la_city_center' column
print(LA_foreclosures_2015_to_2022[['latitude', 'longitude', 'distance_to_la_city_center']])

        latitude   longitude  distance_to_la_city_center
311    33.929188 -118.267830                   13.858041
331    34.166635 -118.558790                   31.677087
332    34.051050 -118.268789                    2.314952
333    33.966885 -118.387358                   16.289111
334    34.196811 -118.601545                   36.654745
...          ...         ...                         ...
29753  34.211720 -118.561370                   34.198443
29754  34.261570 -118.517800                   34.323468
29755  33.956490 -118.425370                   19.841929
29756  33.840820 -118.303770                   24.148683
29757  34.258210 -118.421570                   28.153456

[28954 rows x 3 columns]


In [53]:
#I want to remove everything in the 'lender' column that is after 'C/O' (and variations of) as well as 'as trustee'.
def clean_lender(cell_value):
    # Define the list of patterns to remove
    patterns = ['c/o', '/o', 'c/', ', as trustee', 'as trustee']
    
    # Iterate over each pattern and remove it from the cell value
    for pattern in patterns:
        # Check if the pattern is present in the cell
        if pattern in str(cell_value).lower():
            # Remove everything after the pattern
            cleaned_lender_value = str(cell_value).lower().split(pattern, 1)[0].strip()
            # Update the cell value with the cleaned lender name
            cell_value = cleaned_lender_value
    
    return cell_value

# Apply the clean_lender function to the 'lender' column
LA_foreclosures_2015_to_2022['lender'] = LA_foreclosures_2015_to_2022['lender'].apply(clean_lender)

# Display the updated DataFrame
print(LA_foreclosures_2015_to_2022.head())



            apn registered_date  property_type  \
311  6083008010      01/12/2015   Multi-Family   
331  2164023003      22/01/2015  Single Family   
332  5143012013      22/04/2015  Single Family   
333  4106029005      15/07/2015  Single Family   
334  2138005033      24/08/2015  Single Family   

                                      property_address property_city  \
311  346 E 116TH ST\nLOS ANGELES, CA 90061\n(33.929...   LOS ANGELES   
331  19531 W COLLIER ST\nLOS ANGELES, CA 91356\n(34...   LOS ANGELES   
332  1329 W LINWOOD AVE\nLOS ANGELES, CA 90017\n(34...   LOS ANGELES   
333  7870 S BOEING AVE\nLOS ANGELES, CA 90045\n(33....   LOS ANGELES   
334  6941 OWENSMOUTH AVE\nCANOGA PARK, CA 91303\n(3...   CANOGA PARK   

    property_state  property_zip  council_district  \
311             CA       90061.0                15   
331             CA       91356.0                 3   
332             CA       90017.0                 1   
333             CA       90045.0                11

In [54]:
#I only want the unique values of lenders that have a 60% higher string match

"""# clean and capitlize values to ease analysis
cleaned_lender_values = LA_foreclosures_2015_to_2022['lender'].apply(lambda x: x.upper())

def calculate_similarity(string1, string2):
    distance = Levenshtein.distance(string1, string2)
    max_length = max(len(string1), len(string2))

    # Check if max_length is zero to avoid division by zero
    if max_length == 0:
        return 0.0

    similarity = 1 - distance / max_length
    return similarity

def find_similar_lenders_for_all(all_lenders, threshold=0.6):
    all_similar_lenders = {}

    for i, target_lender in enumerate(all_lenders):
        unique_similar_lenders = set()

        for j, other_lender in enumerate(all_lenders):
            if i != j:  # Exclude comparing the lender to itself
                similarity = calculate_similarity(target_lender, other_lender)
                if similarity >= threshold:
                    unique_similar_lenders.add(other_lender)

        all_similar_lenders[target_lender] = list(unique_similar_lenders)

    return all_similar_lenders

# Find similar lenders for all lenders in the cleaned 'lender' column
all_similar_lenders = find_similar_lenders_for_all(cleaned_lender_values)

# Display the results
for target_lender, similar_lenders in all_similar_lenders.items():
    print(f"Target Lender: {target_lender}")
    print("Similar Lenders:")
    for lender in similar_lenders:
        print(lender)
    print()"""

'# clean and capitlize values to ease analysis\ncleaned_lender_values = LA_foreclosures_2015_to_2022[\'lender\'].apply(lambda x: x.upper())\n\ndef calculate_similarity(string1, string2):\n    distance = Levenshtein.distance(string1, string2)\n    max_length = max(len(string1), len(string2))\n\n    # Check if max_length is zero to avoid division by zero\n    if max_length == 0:\n        return 0.0\n\n    similarity = 1 - distance / max_length\n    return similarity\n\ndef find_similar_lenders_for_all(all_lenders, threshold=0.6):\n    all_similar_lenders = {}\n\n    for i, target_lender in enumerate(all_lenders):\n        unique_similar_lenders = set()\n\n        for j, other_lender in enumerate(all_lenders):\n            if i != j:  # Exclude comparing the lender to itself\n                similarity = calculate_similarity(target_lender, other_lender)\n                if similarity >= threshold:\n                    unique_similar_lenders.add(other_lender)\n\n        all_similar_lenders

In [55]:
# Capitalize all values in the 'lender' column
LA_foreclosures_2015_to_2022['lender'] = LA_foreclosures_2015_to_2022['lender'].str.upper()

# Display the updated DataFrame
print(LA_foreclosures_2015_to_2022['lender'])


311           WELLS FARGO BANK N.A.
331              JP MORGAN CHASE NA
332                   DEUTSCHE BANK
333                 BANK OF AMERICA
334      SELECT PORTFOLIO SERVICING
                    ...            
29753                  LOANCARE LLC
29754                  LOANCARE LLC
29755                    CENLAR FSB
29756      PLANET HOME LENDING, LLC
29757           WELL FARGO BANK N.A
Name: lender, Length: 28954, dtype: object


In [56]:
#renaming the lenders for uniformity

lender_mapping = {
    'WELLS FARGO BANK, N.A.': 'WELLS FARGO BANK',
    'WELLS FARGO BANK NA': 'WELLS FARGO BANK',
    'WELLS FARGO BANK NA,': 'WELLS FARGO BANK',
    'WELLS FARGO BANK, N.A.,': 'WELLS FARGO BANK',
    'WELLS FARGO BANK N.A.': 'WELLS FARGO BANK',
    'WELLS FARGO BANK, N.A.': 'WELLS FARGO BANK',
    'WELLS FARGO BANK, N.A': 'WELLS FARGO BANK',
    'WELL FARGO BANK N.A': 'WELLS FARGO BANK',
    'WELLS FARGO BANK, NA,': 'WELLS FARGO BANK',
    'WELLS FARGO BANK N.A': 'WELLS FARGO BANK',
    'WELLS FARGO BANK, NA': 'WELLS FARGO BANK',
    'WELLS FARGO': 'WELLS FARGO BANK',
    'WELLS FARGO BANK, NATIONAL ASSOCIATION': 'WELLS FARGO BANK',
    'JP MORGAN CHASE BANK, N.A.': 'JP MORGAN CHASE',
    'JPMORGAN CHASE BANK N.A.': 'JP MORGAN CHASE',
    'JP MORGAN CHASE NA': 'JP MORGAN CHASE',
    'JP MORGAN CHASE BK': 'JP MORGAN CHASE',
    'JPMORGAN CHASE BANK, N.A.': 'JP MORGAN CHASE',
    'JP MOGAN CHASE': 'JP MORGAN CHASE',
    'JPMORGAN CHASE BANK': 'JP MORGAN CHASE',
    'JPMORGAN CHASE BANK, NA': 'JP MORGAN CHASE',
    'JP MORGAN CHASE BK': 'JP MORGAN CHASE',
    'DEUTSCHE BANK N.T.C.': 'DEUTSCHE BANK',
    'DEUTSCHE BANK': 'DEUTSCHE BANK',
    'DEUTSCHE BANK N.A.': 'DEUTSCHE BANK',
    'DEUTSCHE BANK NA': 'DEUTSCHE BANK',
    'DEUTSCHE BANK,': 'DEUTSCHE BANK',
    'DEUTSCHE BANK NATIONAL TRUST': 'DEUTSCHE BANK',
    'DEUTSCHE BK NATL TRUST CO': 'DEUTSCHE BANK',
    'DEUTSCHE BANK NATIONAL TRUST  COMPANY': 'DEUTSCHE BANK',
    'DEUTSCHE BANK NATIONAL TRUST,': 'DEUTSCHE BANK',
    'DEUTSCHE BANK NATIONAL TRUST': 'DEUTSCHE BANK',
    'DEUTSH BANK NATIONAL TRUST': 'DEUTSCHE BANK',
    'DEUTSCHE BANK NATIONAL TRUST COMPANY': 'DEUTSCHE BANK',
    'DEUTSCHE BANK NATIONAL TRUST  COMPANY': 'DEUTSCHE BANK',
    'DEUTSCHE BANK NATIONAL TRUST,': 'DEUTSCHE BANK',
    'DEUTSCHE BANK NATIONAL TRUST': 'DEUTSCHE BANK',
    'DEUTSH BANK NATIONAL TRUST': 'DEUTSCHE BANK',
    'DEUTSCHE BANK NATIONAL TRUST COMPANY': 'DEUTSCHE BANK',
    'DEUTSCHE BANK TRUST CO AMERICAS': 'DEUTSCHE BANK',
    'DEUTSCHE BANK NATIONAL TRUST  CO SELECT PORTFOLIO SERVICING': 'DEUTSCHE BANK',
    'DEUTSCHE BANK NATIONAL TRUST CO SELECT PORTFOLIO SERVICING': 'DEUTSCHE BANK',
    'DEUTSCHE BANK NATIONAL TRUST CO SELECT PORFOLIO SERVICING': 'DEUTSCHE BANK',
    'DEUTSCHE BANK NATIONAL TRUST  COMPANY': 'DEUTSCHE BANK',
    'BANK OF AMERICA - PRE': 'BANK OF AMERICA',
    'BANK OF AMERICA NA': 'BANK OF AMERICA',
    'SELECT PORTFOLIO SERVICING LLC': 'SELECT PORTFOLIO SERVICING',
    'SELECT PORTFOLIO SERVICING, INC.': 'SELECT PORTFOLIO SERVICING',
    'SELECT PORTFOLIO SERVICING INC': 'SELECT PORTFOLIO SERVICING',
    'SELECT PORTFOLIO SERVICING, INC': 'SELECT PORTFOLIO SERVICING',  
    'SELECT PORTFOLIO SERVICING INC': 'SELECT PORTFOLIO SERVICING',
    'SELECT PORTFOLIO SERVICING': 'SELECT PORTFOLIO SERVICING',
    'SENECA MORTGAGE SERVICING LLC': 'SELECT PORTFOLIO SERVICING',
    'SELECT PORTFOLIO SERVICING LLC': 'SELECT PORTFOLIO SERVICING',
    'SELECT PORTFOLIO SERVICING, INC': 'SELECT PORTFOLIO SERVICING',
    'SELECT PORTFOLIO SERVICING, INC.': 'SELECT PORTFOLIO SERVICING',
    'SELECT PORTFOLIO SERVICING INC': 'SELECT PORTFOLIO SERVICING',
    'OCWEN LOAN SERVICING, LLC':'OCWEN LOAN SERVICING',
    'OCWEN LOAN SERVICING LLC':'OCWEN LOAN SERVICING',
    'OCWEN LOAN SERVICING, LLC':'OCWEN LOAN SERVICING',
    'OCWEN LOAN SERVICING,LLC':'OCWEN LOAN SERVICING',
    'OCWEN FINANCIAL CORPORATION':'OCWEN LOAN SERVICING',
    'M & T BANK': 'M&T BANK',
    'M&T BANK': 'M&T BANK',
    'CAPITAL ONE, NA': 'CAPITAL ONE',
    'NATIONSTAR MORTGAGE LLC': 'NATIONSTAR MORTGAGE',
    'NATIONS DIRECT MORTGAGE, LLC': 'NATIONSTAR MORTGAGE',
    'NATIONS DIRECT MORTGAGE LLC,': 'NATIONSTAR MORTGAGE',
    'NATIONS DIRECT MORTGAGE': 'NATIONSTAR MORTGAGE',
    'NATIONS DIRECT MORTGAGE LLC': 'NATIONSTAR MORTGAGE',
    'NATIONSTAR MORTGAGE. LLC': 'NATIONSTAR MORTGAGE',
    'NATIONSTAR MORTGAGE, LLC': 'NATIONSTAR MORTGAGE',
    'MR. COOPER/NATIONSTAR MORTGAGE, LLC.': 'NATIONSTAR MORTGAGE',
    'BANK OF NEW YORK MELLON TRUST': 'BANK OF NEW YORK MELLON',
    'BANK OF NEW YORK': 'BANK OF NEW YORK MELLON',
    'THE BANK OF NEW YORK MELLO': 'BANK OF NEW YORK MELLON',
    'THE BANK OF NEW YORK MELLON,': 'BANK OF NEW YORK MELLON',
    'THE BANK OF NEW YORK MELLON': 'BANK OF NEW YORK MELLON',
    'BANK OF NEW YORK MELLON,': 'BANK OF NEW YORK MELLON',
    'THE BANK OF NEW YORK MELLON FKA THE BANK OF NEW YORK': 'BANK OF NEW YORK MELLON',
    'THE BANK OF NEW YORK MELLON, FKA, THE BANK OF NEW YORK': 'BANK OF NEW YORK MELLON',
    'BANK OF NY MELLON 2005-32T': 'BANK OF NEW YORK MELLON',
    'THE BANK OF NEW YORK MELLON, DITECH FINANCIAL LLC': 'BANK OF NEW YORK MELLON',
    'SELENE FINANCE LP': 'SELENE FINANCE',
    'SELENE FINANCING': 'SELENE FINANCE',
    'BSI FINANCIAL SERVICES, INC.': 'BSI FINANCIAL SERVICES',
    'BSI FINANCIAL SERVICES, INC.': 'BSI FINANCIAL SERVICES',
    'BSI FINANCIAL': 'BSI FINANCIAL SERVICES',
    'BSI': 'BSI FINANCIAL SERVICES',
    'REVERSE MORTGAGE SOLUTIONS, INC.': 'REVERSE MORTGAGE SOLUTIONS',
    'REVERSE MORTGAGE SOLUTIONS, INC.': 'REVERSE MORTGAGE SOLUTIONS',
    'REVERSE MORTGAGE SOLUTIONS INC.': 'REVERSE MORTGAGE SOLUTIONS',
    'REVERSE MORTGAGE FUNDING LLC': 'REVERSE MORTGAGE SOLUTIONS',
    'REVERSE MORGAGE SOLUTIONS': 'REVERSE MORTGAGE SOLUTIONS',
    'REVERSE MORTGAGE FUNDING': 'REVERSE MORTGAGE SOLUTIONS',
    'SHELLPOINT MORTGAGE SERVICES': 'SHELLPOINT MORTGAGE',
    'SHELLPOINT MORTGAGE SERVICES': 'SHELLPOINT MORTGAGE',
    'SHELLPOINT MORTGAGE SERVICING': 'SHELLPOINT MORTGAGE',
    'CITIMORTGAGE': 'CITIBANK',
    'CITIMORTGAGE, INC.': 'CITIBANK',
    'CITIMORTGAGE': 'CITIBANK',
    'CITI MORTGAGE': 'CITIBANK',
    'CIT BANK, N.A.': 'CITIBANK',
    'CITIBANK, N.A.': 'CITIBANK',
    'CITIBANK, N.A.,': 'CITIBANK',
    'CITIBANK, N.A': 'CITIBANK',
    'CITIBANK, N.A.': 'CITIBANK',
    'CIT BANK, N.A.': 'CITIBANK',
    'CITIBANK, N.A.,': 'CITIBANK',
    'CITIBANK NA': 'CITIBANK',
    'CITI BANK NA': 'CITIBANK',
    'CITIMORTAGAE, INC': 'CITIBANK',
    'CIT': 'CITIBANK',
    'CITIMORTGAGE, INC.': 'CITIBANK',
    'CITIMORTAGAE, INC': 'CITIBANK',
    'FINANCIAL FREEDOM, A DIVISION OF CIT BANK': 'CITIBANK',
    'FINANCIAL FREEDOM, A DIVISION OF CIT BANK.': 'CITIBANK',
    'FINANCIAL FREEDOM A DIVISION OF CIT BANK NA': 'CITIBANK',
    'CARRINGTON REAL ESTATE SERVICES': 'CARRINGTON MORTGAGE SERVICES',
    'CARRINGTON MORTGAGE SERVICES LLC': 'CARRINGTON MORTGAGE SERVICES',
    'CARRINGTON MORTGAGE SERVICES, LLC': 'CARRINGTON MORTGAGE SERVICES',
    'CARRINGTON MORTGAGE SERVICES LLC': 'CARRINGTON MORTGAGE SERVICES',
    'HSBC CONSUMER LENDING': 'HSBC',
    'HSBC BANK USA, N.A.,': 'HSBC',
    'HSBC BANK USA N.A.': 'HSBC',
    'HSBC BANK USA, N.A.': 'HSBC',
    'HSBC BANK USA,  N.A.': 'HSBC',
    'HSBC BANK USA CO SELECT PORTFOLIO SERVICING': 'HSBC',
    'HSBC MORTGAGE CORPORATION': 'HSBC',
    'HSBC MORTGAGE SERVICES': 'HSBC',
    'HSBC BANK USA NATIONAL ASSOCIATION': 'HSBC',
    'HSBC BANK USA, NATIONAL ASSOCIATION': 'HSBC',
    'HSBC BANK USA': 'HSBC',
    'HSBC BANK USA,': 'HSBC',
    'HSBC BANK USA, N.A.,': 'HSBC',
    'HSBC BANK USA N.A.': 'HSBC',
    'HSBC BANK USA,  N.A.': 'HSBC',
    'HSBC BANK USA NA': 'HSBC',
    'HSBC BANK USA, N.A.': 'HSBC',
    'HSBC CONSUMER LENDING MORTGAGE SERVICES': 'HSBC',
    'HSBC BANK USA CO SELECT PORTFOLIO SERVICING'
    'US BANK, N.A.': 'U.S. BANK',
    'U.S BANK, N.A.': 'U.S. BANK',
    'US BANK N.A.': 'U.S. BANK',
    'US BANK NA': 'U.S. BANK',
    'U.S. BANK, N.A.': 'U.S. BANK',
    'U.S. BANK, N.A': 'U.S. BANK',
    'U.S. BANK, N.A.,': 'U.S. BANK',
    'U.S. BANK NA,': 'U.S. BANK',
    'U.S BANK N.A': 'U.S. BANK',
    'U.S. BANK, NA': 'U.S. BANK',
    'US BANK, N.A.': 'U.S. BANK',
    'U.S. BANK NA': 'U.S. BANK',
    'US BANK NA,': 'U.S. BANK',
    'U.S. BANK,  N.A.': 'U.S. BANK',
    'U.S. BANK N.A.,': 'U.S. BANK',
    'U.S. BANK N.A.': 'U.S. BANK',
    'U.S. BANK/RMA': 'U.S. BANK', 
    'US BANK NA CO SELECT PORTFOLIO SERVICING': 'U.S. BANK',
    'U.S. BANK N.A CO SELECT PORTFOLIO SERVICING': 'U.S. BANK',
    'US BANK HOME MORTGAGES': 'U.S. BANK',
    'US BANK HOME MORTGAGE': 'U.S. BANK',
    'U.S. BANK HOME MORTGAGE': 'U.S. BANK',
    'US BANK HOME MORTGAGES': 'U.S. BANK',
    'US BANK NATIONAL ASSN': 'U.S. BANK',
    'U.S. BANK NATIONAL ASSOCIATION': 'U.S. BANK',
    'US-BANK NATIONAL ASSOCIATION': 'U.S. BANK',
    'U.S. BANK NATIONAL ASSOCIATION': 'U.S. BANK',
    'US BANK NATIONAL ASSN': 'U.S. BANK',
    'US BANK NATIONAL ASSOCIATION': 'U.S. BANK',
    'U.S. BANK NATIONAL ASSOCIATION / BLACKSTONE': 'U.S. BANK',
    'U.S BANK NATIONAL ASSOCIATION': 'U.S. BANK',
    'U.S. BANK, NATIONAL ASSOCIATION': 'U.S. BANK',
    'U.S. BANK NATIONAL ASSOCIATION,': 'U.S. BANK',
    'U.S. BANK  NATIONAL ASSOCIATION': 'U.S. BANK',
    'US BANK HOME MORTGAGE': 'U.S. BANK',
    'US BANK HOME MORTGAGE': 'U.S. BANK',
    'U.S. BANK HOME MORTGAGE': 'U.S. BANK',
    'US BANK HOME MORTGAGES': 'U.S. BANK',
    'US BANK HOME MORTGAGE': 'U.S. BANK',
    'US BANK HOME MORTGAGES': 'U.S. BANK',
    'U.S. BANK NATIONAL ASSOCIATION, AS INDENTURE TRUSTEE, ON BEHALF OF THE HOLDERS OF THE TERWIN MORTGAG': 'U.S. BANK',
    'U.S. BANK NATIONAL ASSOCIATION, AS INDENTURE TRUSTEE ON BEHALF OF AND WITH RESPECT TO AJAX MORTGAGE': 'U.S. BANK',
    'US BANK NA CO SELECT PORTFOLIO SERVICING': 'U.S. BANK',
    'U.S. BANK, N.A. CO SELECT PORTFOLIO SERVICING': 'U.S. BANK',
    'U.S. BANK N.A CO SELECT PORTFOLIO SERVICING': 'U.S. BANK',
    'US BANK NA SERIES 2007-AR7': 'U.S. BANK',
    'U.S. BANK, NA AS LEGAL TITLE TRUSTEE FOR TRUMAN 2013 SC4 TITLE TRUST': 'U.S. BANK',
    'U.S. BANK, NA AS LEGAL TITLE TRUSTEE FOR TRUMAN 2013 SC4 TITLE TRUST': 'U.S. BANK',
    'U.S. BANK TRUSTEE - TRUMAN 2016 SC6 TITLE TRUST': 'U.S. BANK',
    'U.S. BANK, NATIONAL ASSOCIATION AS LEGAL TITLE TRUSTEE FOR TRUMAN 2016 SC6 TITLE TRUST': 'U.S. BANK',
    'U S BANK NA AS TR OF TRUMAN 2016 SC6 TITLE TRUST': 'U.S. BANK',
    'U.S. BANK, NA AS LEGAL TITLE TRUSTEE': 'U.S. BANK',
    'U.S. BANK, NA AS LEGAL TITLE TRUSTEE FOR TRUMAN 2013 SC4 TITLE TRUST': 'U.S. BANK',
    'U.S. BANK, NATIONAL ASSOCIATION AS LEGAL TITLE TRUSTEE FOR TRUMAN 2016 SC6 TITLE TRUST': 'U.S. BANK',
    'CALIBER HOME LOAN': 'CALIBER HOME LOANS',
    'CALIBER HOME LOANS, INC.': 'CALIBER HOME LOANS',
    'CALIBER HOME LOANS': 'CALIBER HOME LOANS',
    'CALIBER HOME LOANS, INC': 'CALIBER HOME LOANS',
    'CHAMPION HOME LOANS': 'CALIBER HOME LOANS',
    'CALIBER HOME LOANS INC': 'CALIBER HOME LOANS',  
    'PENNY MAC LOAN SERVICE': 'PENNY MAC',
    'PENNYMAC LOAN SERVICES LLC': 'PENNY MAC',
    'PENNYMAC LOAN SERVICES, LLS': 'PENNY MAC',
    'PENNY MAC LOAN SERVICE': 'PENNY MAC',
    'PENNYMAC LOAN SERVICES': 'PENNY MAC',
    'PENNYMAC LOAN SERVICES, LLC': 'PENNY MAC',
    'FEDERAL NATIONAL MORTGAGE ASSOCIATION': 'FEDERAL NATIONAL MORTGAGE',
    'FEDERAL NATIONAL MORTGAGE ASS RECONTRUST COMPANY': 'FEDERAL NATIONAL MORTGAGE',
    'FEDERAL NATIONAL MORTGAGE ASSOCIATION': 'FEDERAL NATIONAL MORTGAGE',
    'FEDERAL HOME LOAN MORTGAGE CORPORATION': 'FEDERAL NATIONAL MORTGAGE',
    'FEDERAL NATIONAL MORTGAGE FNMA': 'FEDERAL NATIONAL MORTGAGE',
    'FEDERAL NATIONAL MORTGAGE ASSOCIATION - REVERSE  MORTGAGE SOLUTIONS INC': 'FEDERAL NATIONAL MORTGAGE',
    'FEDERAL HOME LOAN MORTGAGE': 'FEDERAL NATIONAL MORTGAGE',
    'FEDERAL NATIONAL MORTGAGE ASSOCIATION': 'FEDERAL NATIONAL MORTGAGE',
    'FEDERAL NATIONAL MORTGAGE FNMA': 'FEDERAL NATIONAL MORTGAGE',
    'FEDERAL NATIONAL MORTGAGE ASS RECONTRUST COMPANY': 'FEDERAL NATIONAL MORTGAGE',
    'FEDERAL NATIONAL MORTGAGE ASSOCIATION': 'FEDERAL NATIONAL MORTGAGE',
    'FEDERAL NATIONAL MORTGAGE FNMA': 'FEDERAL NATIONAL MORTGAGE',
    'FEDERAL NATL MTG ASSN FNMA': 'FEDERAL NATIONAL MORTGAGE',
    'SETERUS INC': 'SETERUS',
    'SETERUS, INC': 'SETERUS',
    'SETERUS INC': 'SETERUS',
    'SETERUS INC,': 'SETERUS',
    'WELLS FARGO BANK, N.A.': 'WELLS FARGO BANK',
    'WELLS FARGO BANK, N.A.': 'WELLS FARGO BANK',
    'WELLS FARGO BANK, N.A': 'WELLS FARGO BANK',
    'WELL FARGO BANK N.A': 'WELLS FARGO BANK',
    'WELLS FARGO BANK, NA,': 'WELLS FARGO BANK',
    'WELLS FARGO BANK N.A': 'WELLS FARGO BANK',
    'WELLS FARGO BANK, NA': 'WELLS FARGO BANK',
    'WELLS FARGO BANK N.A.': 'WELLS FARGO BANK',
    'WELLS FARGO BANK, N.A.,': 'WELLS FARGO BANK',
    'WELLS FARGO BANK NA,': 'WELLS FARGO BANK',
    'WELLS FARGO BANK NA': 'WELLS FARGO BANK',
    'WELLS FARGO BANK CO SELECT PORTFOLIO SERVICING'
    'DOVENMUEHLE MORTGAGE, INC.': 'DOVENMUEHLE MORTGAGE',
    'DOVENMUEHLE MORTGAGE INC.': 'DOVENMUEHLE MORTGAGE',
    'DOVENMUEHLE MORTGAGE, INC.': 'DOVENMUEHLE MORTGAGE',
    'DOVENMUEHLE MORTGAGE, INC': 'DOVENMUEHLE MORTGAGE',
    'SPECIALIZED ASSET MANAGEMENT LLC': 'SPECIALIZED ASSET MANAGEMENT',
    'CENTER FSB,': 'CENLAR FSB',
    'CENLAR FSB,': 'CENLAR FSB',
    'CENLAR FSB': 'CENLAR FSB',
    'MUFG UNION BANK, N.A.': 'MUFG UNION BANK',
    'PENNYMAC': 'PENNY MAC',
    'PENNYMAC LOAN SERVICES': 'PENNY MAC',
    'PENNYMAC LOAN SERVICES LLC': 'PENNY MAC',
    'PENNYMAC LOAN SERVICES, LLS': 'PENNY MAC',
    'PENNY MAC LOAN SERVICE': 'PENNY MAC',
    'PENNYMAC LOAN SERVICES': 'PENNY MAC',
    'PENNYMAC LOAN SERVICES, LLC': 'PENNY MAC',
    'PENNYMAC CORP': 'PENNY MAC', 
    'PENNYMAC HOLDINGS, LLC': 'PENNY MAC', 
    'PENNYMAC LOAN SERVICES, LLC': 'PENNY MAC', 
    'PNC MORTGAGE PRESALE': 'PNC',
    'PNC MORTGAGE': 'PNC',
    'PNC BANK, N.A.': 'PNC',
    'PNC BANK, NATIONAL ASSOCIATION,': 'PNC',
    'PNC BANK, NATIONAL ASSOCIATION': 'PNC',   
    'PNC BANK': 'PNC',
    'PNC BANK-PNCB': 'PNC',
    'PNC BANK': 'PNC',
    'PNC BANK,': 'PNC',
    'PNC BANK,': 'PNC',
    'PNC BANK-PNCB': 'PNC',
    'PNC BANK': 'PNC',
    'PNC BANK, N.A.': 'PNC',  
    'SUN WEST MORTGAGE COMPANY': 'SUN WEST MORTGAGE',
    'SUNWEST MORTGAGE': 'SUN WEST MORTGAGE',
    'US DEPT OF HOUSING AND URBAN DEVELOPMENT': 'HUD',
    'US DEPT OF HOUSING AND URBAN DEVELOPMENT': 'HUD',
    'ONEWEST BANK NA': 'ONEWEST BANK',
    'ONE WEST BANK': 'ONEWEST BANK',
    'SPECIALIZED LOAN SERVICING AS SERVICING AGENT FOR BENEFICIARY': 'SPECIALIZED LOAN SERVICING',
    'SPECIALIZED LOAN SERVICING LLC ON BEHALF OF BENEFICIARY'
    'GREEN TREE SERVICING LLC': 'GREEN TREE SERVICING',
    'GREE TREE SERVICING LLC': 'GREEN TREE SERVICING',
    'GREEN TREE': 'GREEN TREE SERVICING',
    'PACIFIC UNION FINANCIAL, LLC': 'PACIFIC UNION FINANCIAL',
    'RUSHMORE LOAN MANAGEMENT SERVICES': 'RUSHMORE LOAN MANAGMENT',
    'RUSHMORE LOAN MANAGEMENT SERVICES LLC': 'RUSHMORE LOAN MANAGMENT',
    'RUSHMORE LOAN MGMT SERVICES': 'RUSHMORE LOAN MANAGMENT',
    'RUSHMORE LOAN MANAGEMENT': 'RUSHMORE LOAN MANAGMENT',
    'RESIDENTIAL CREDIT OPPORTUNITY V': 'RESIDENTIAL CREDIT SOLUTIONS',
    'RESIDENTIAL CREDIT SOLUTION': 'RESIDENTIAL CREDIT SOLUTIONS',
    'RESIDENTIAL CREDIT SOLUTIONS, INC.': 'RESIDENTIAL CREDIT SOLUTIONS',
    'BAYVIEW LOAN SERVICEING, LLC': 'BAYVIEW LOAN SERVICING',
    'BAYVIEW LOAN SERVICING JEFFERSON': 'BAYVIEW LOAN SERVICING',
    'BAYVIEW LOAN SEVICING, LLC': 'BAYVIEW LOAN SERVICING',
    'BAYVIEW LOAN SERVICING': 'BAYVIEW LOAN SERVICING',
    'BAYVIEW LOAN SERVICING REO': 'BAYVIEW LOAN SERVICING',
    'BAYVIEW LOAN SERVICING, LLC': 'BAYVIEW LOAN SERVICING',
    'BAYVIEW LOAN SERVICING LLC': 'BAYVIEW LOAN SERVICING',
    'BAYVIEW LOAN SERVICING LLC FOR BANK OF NEW YORK MELLON': 'BAYVIEW LOAN SERVICING',
    'BAYVIEW LOAN SERVICING/BAYVIEW REPRESENTATIVE': 'BAYVIEW LOAN SERVICING',
    'BAYVIEW LOAN SERVICING JEFFERSON': 'BAYVIEW LOAN SERVICING',
    'CHAMPION MORTGAGE COMPANY': 'CHAMPION MORTGAGE',
    'CHAMPION HOME LOANS': 'CHAMPION MORTGAGE',
    'CHAMPION MORTGAGE COMPANY NATIONSTAR MORTGAGE LLC, DBA': 'CHAMPION MORTGAGE',  
    'PLM LOAN MANAGEMENT SERVICES, INC': 'PLM LOAN MANAGEMENT SERVICES', 
    'PLM LOAN MANAGEMENT SERVICES, INC.':'PLM LOAN MANAGEMENT SERVICES',
    'STATEBRIDGE CO': 'STATEBRIDGE',
    'STATEBRIDGE COMPANY LLC': 'STATEBRIDGE',
    'PHH MORTGAGE CORP': 'PHH MORTGAGE',
    'PHH MORTGAGE SERVICES': 'PHH MORTGAGE',
    'PHH MORTGAGE CORPORATION': 'PHH MORTGAGE',
    'PHH MORGAGE CORPORATION': 'PHH MORTGAGE',
    'PHH MORTGAGE CORPORATION': 'PHH MORTGAGE',
    'PHH MORTGAGE CORP': 'PHH MORTGAGE',   
    'O.K.,LLC': 'OK LLC',
    'HUD-HOUSING/URBAN DEV': 'HUD',
    'PHH MORTGAGE CORP F/K/A CENDANT MORTGAGE CORP': 'PHH MORTGAGE CORP',
    'GUILD MORTGAGE COMPANY': 'GUILD MORTGAGE',
    'CELINK/MAM': 'CELINK',
    'WILMINGTON TRUST CO': 'WILMINGTON TRUST',
    'WILMINGTON TRUST CO': 'WILMINGTON TRUST',
    'WILMINGTON TRUST NA': 'WILMINGTON TRUST',
    'WILMINGTON TRUST N.A.': 'WILMINGTON TRUST',
    'WILMINGTON TRUST,': 'WILMINGTON TRUST',
    'WILMINGTON TRUST, NA': 'WILMINGTON TRUST',
    'WILMINGTON TRUST, NA,': 'WILMINGTON TRUST',
    'WILMINGTON TRUST NA,': 'WILMINGTON TRUST',
    'LOANCARE LLC': 'LOANCARE',
    'LOANCARE, LLC': 'LOANCARE',
    'LOANCARE, A DIVISION OF FNF SERVICING, INC': 'LOANCARE',
    'DITECH FINANCIAL LLC': 'DITECH FINANCIAL',
    'DITECH FINANCIAL, LLC': 'DITECH FINANCIAL',
    'SUNWEST MORTGAGE': 'SUN WEST MORTGAGE',
    'SUN WEST MORTGAGE COMPANY': 'SUN WEST MORTGAGE',
    'FAY SERVICING, LLC': 'FAY SERVICING',
    'FAY SERVICING LLC': 'FAY SERVICING',
    'FREEDOM MORTGAGE CORP': 'FREEDOM MORTGAGE',
    'FREEDOM MORTGAGE CORPORATION': 'FREEDOM MORTGAGE',
    'CLEARSPRINGLS': 'CLEARSPRINGS LS',
    'EAST WEST BANK': 'EAST-WEST BANK',
    '21ST  MORTGAGE CORPORATION': '21ST MORTGAGE CORPORATION',
    'DLJ MORTGAGE CAPITAL, INC.,': 'DLJ MORTGAGE CAPITAL',
    'DLJ MORTGAGE CAPITAL, INC': 'DLJ MORTGAGE CAPITAL',
    'DLJ MORTGAGE CAPITAL,': 'DLJ MORTGAGE CAPITAL',
    'DLJ MORTGAGE CAPITAL, INC.': 'DLJ MORTGAGE CAPITAL',
    'NATIONS DIRECT MORTGAGE, LLC': 'NATIONS DIRECT MORTGAGE',
    'NATIONS DIRECT MORTGAGE LLC,': 'NATIONS DIRECT MORTGAGE',
    'NATIONS DIRECT MORTGAGE LLC': 'NATIONS DIRECT MORTGAGE',
    'NATIONSTAR MORTGAGE. LLC': 'NATIONS DIRECT MORTGAGE',
    'NATIONSTAR MORTGAGE, LLC': 'NATIONS DIRECT MORTGAGE',
    'THE EVERGREEN ADVANTAGE, LLC': 'EVERGREEN ADVANTAGE',
    'EVERGREEN ADVANTAGE, LLC': 'EVERGREEN ADVANTAGE',
    'FLAGSTAR BANK': 'FLAGSTAR',
    'SHELLPOINT MORTGAGE SERVICING': 'SHELLPOINT MORTGAGE',
    'SHELLPOINT MORTGAGE SERVICES': 'SHELLPOINT MORTGAGE',
    'PMAC LENDING SERVICES, INC.': 'PMAC LENDING SERVICES',
    'PMAC LENDING SERVICES, INC': 'PMAC LENDING SERVICES',
    'PROVIDENT FUNDING ASSOCIATES LP': 'PROVIDENT FUNDING ASSOCIATES',
    'PROVIDENT FUNDING ASSOCIATES L.P.': 'PROVIDENT FUNDING ASSOCIATES',
    'PROVIDENT FUNDING ASSOCIATES, L.P.': 'PROVIDENT FUNDING ASSOCIATES',
    'PROVIDENT FUNDING': 'PROVIDENT FUNDING ASSOCIATES',
    'WEST COAST CAPITAL, LLC': 'WEST COAST CAPITAL',
    'WEST COAST CAPITAL GROUP, INC.': 'WEST COAST CAPITAL',
    'STARLIGHT CAPITAL GROUP, LLC': 'WEST COAST CAPITAL',
    'TRINITY FINANCIAL SERVICES, LLC': 'TRINITY FINANCIAL SERVICES',
    'TRINITY FINANCE SERVICES, LLC': 'TRINITY FINANCIAL SERVICES',
    'TRINITY FINANCIAL SERVICES LLC': 'TRINITY FINANCIAL SERVICES',
    'TRINITY FINANCE SERVICES, LLC': 'TRINITY FINANCIAL SERVICES',
    'TRINITY FINANCIAL SERVICES, LLC': 'TRINITY FINANCIAL SERVICES',
    'TRINITY FINANCE SERVICES, LLC': 'TRINITY FINANCIAL SERVICES',
    'TRINITY FINANICAL SERVICE, LLC': 'TRINITY FINANCIAL SERVICES',
    'TRINITY FINANCIAL SERVICES': 'TRINITY FINANCIAL SERVICES',
    'TRINITY FINANCIAL SERVICES LLC': 'TRINITY FINANCIAL SERVICES',
    'TRINITY': 'TRINITY FINANCIAL SERVICES',
    'TRUSTEE CORP': 'TRUSTEE CORPS',
    'PARK TREE INVESTMENTS 21, LLC': 'PARK TREE INVESTMENTS',
    'QUICKEN LOANS INC': 'QUICKEN LOANS',
    'WANG YANG ENTERPRISES, LLC': 'WANG YANG ENTERPRISES',
    'WANG YANG ENTERPRISES LLC': 'WANG YANG ENTERPRISES',
    'CENTER STREET LENDING CORP': 'CENTER STREET LENDING',
    'ARAN INVESTMENT INC': 'ARAN INVESTMENTS INC',
    'GUSS INVESTMENT GROUP, LLC': 'GUSS INVESTMENT GROUP',
    'WEST COAST SERVICING INC.': 'WEST COAST SERVICING',
    'WEST COAST SERVICING INC': 'WEST COAST SERVICING',
    'GDARTLLC': 'GDART L.L.C',
    'SABADELL UNITED BANK, N.A.': 'SABABELL UNITED BANK',
    'SABADELL UNITED BANK, N.A.': 'SABABELL UNITED BANK',
    'SABABELL': 'SABABELL UNITED BANK',
    'CHASE HOME FINANCE REO': 'CHASE BANK',
    'CHASE HOME FINANCE': 'CHASE BANK',
    'CHRISTIANA TRUST,': 'CHRISTIANA TRUST',
    'PREMIER AMERICA CU': 'PREMIER AMERICA CREDIT UNION',
    'SUNRISE 111 LLC': 'SUNRISE 111',
    'SUNRISE 111, LLC': 'SUNRISE 111',
    'TRTUSTEE CORPS, M&T BAYVIEW': 'TRUSTEE CORPS',
    'TRUSTEE CORPS - IRVINE': 'TRUSTEE CORPS',
    'TRUSTEE CORPS, PENNYMAC LOAC SERVICES, LLC': 'TRUSTEE CORPS',
    'TRUSTEE CORPS, BAYVIEW LOAN SERVICING': 'TRUSTEE CORPS',
    'TRUSTEE CORPS, PENNYMAC LOAC SERVICES, LLC': 'TRUSTEE CORPS',
    'STATEBRIDGE COMPANY, LLC': 'STATEBRIDGE',
    'STATEBRIDGE CO': 'STATEBRIDGE',
    'STATEBRIDGE COMPANY LLC': 'STATEBRIDGE',
    'MOR FINANCIAL SERVICES INC': 'MOR FINANCIAL SERVICES',
    'AMS-SENECA MORTGAGE SERVICING LLC': 'SENECA MORTGAGE SERVICING LLC',
    'TROJAN CAPITAL INVESTMENTS, LLC': 'TROJAN CAPITAL INVESTMENTS',
    'TROJAN CAPITAL INVESTMENT, LLC': 'TROJAN CAPITAL INVESTMENTS',
    'MIDFIRST BANK, AS SERVICER FOR US BANK NATIONAL ASSOCIATION': 'MIDFIRST BANK',
    'MIDFIRST BANK, AS SERVICER FOR DEUTSCHE BANK NATIONAL TRUST COMPANY': 'MIDFIRST BANK',
    'MIDFIRST BANK, AS SERVICER FOR US BANK NATIONAL ASSOCIATION': 'MIDFIRST BANK',
    'MIDFIRST BANK, AS SERVICER FOR HSBC BANK USA': 'MIDFIRST BANK',
    'THE MONEY SOURCE INC': 'THE MONEY SOURCE',
    'THE MONEY SOURCE INC.': 'THE MONEY SOURCE',
    'THE MONEY SOURCE, INC.': 'THE MONEY SOURCE',
    'THE MONEY SOURCE': 'THE MONEY SOURCE',
    'THE MONEY SOURCE INC,': 'THE MONEY SOURCE',
    'FIRST BANK D/B/A FIRST BANK MORTGAGE': 'FIRST BANK',
    'QUICKEN LOANS INC': 'QUICKEN LOANS',
    'QUICKEN LOAN SERVICING': 'QUICKEN LOANS',
    'DAKOTA ASSET SERVICES, LLC': 'DAKOTA ASSET SERVICES',
    'FRANKLIN CREDIT MANAGEMENT CORPORATION': 'FRANKLIN CREDIT MANAGEMENT CORP',
    'FRANKLIN CREDIT MANAGEMENT CORP.': 'FRANKLIN CREDIT MANAGEMENT CORP',
    'WOODLAND HILLS MORTGAGE CORPORATION ISAOA/ATIMA': 'WOODLAND HILLS MORTGAGE BANKING CORPORATION',
    'WOODLAND HILLS MORTGAGE BANKING CORPORATION ISAOA': 'WOODLAND HILLS MORTGAGE BANKING CORPORATION',
    'WILMINGTON SAVINGS FUND SOCIETY, FSB': 'WILIMINGTON SAVINGS FUND SOCIETY',
    'WILMINGTON SAVINGS FUND SOCIETY FSB': 'WILIMINGTON SAVINGS FUND SOCIETY',
    'WILMINGTON SAVINGS FUND SOCIETY, FSB, D/B/A CHRISTIANA TRUST': 'WILIMINGTON SAVINGS FUND SOCIETY',
    'WILMINGTON SAVINGS FUND SOCIETY, FSB, D/B/A CHRISTIANA TRUST': 'WILIMINGTON SAVINGS FUND SOCIETY',
    'WILMINGTON SAVINGS FUND SOCIETY, FSB': 'WILIMINGTON SAVINGS FUND SOCIETY',
    'WILMINGTON SAVINGS FUND SOCIETY FSB, DBA CHRISTIANA TRUST': 'WILIMINGTON SAVINGS FUND SOCIETY',
    'WILIMINGTON TRUST NA, SUCCESSOR TRUSTEE TO CITIBANK NA': 'WILIMINGTON SAVINGS FUND SOCIETY',
    'WILMINGTON TRUST, NA, SUCCESSOR TRUSTEE TO CITIBANK,': 'WILIMINGTON SAVINGS FUND SOCIETY',
    'NATIONS DIRECT MORTGAGE, LLC': 'NATIONS DIRECT MORTGAGE',
    'NATIONS DIRECT MORTGAGE, LLC': 'NATIONS DIRECT MORTGAGE',
    'NATIONS DIRECT MORTGAGE LLC,': 'NATIONS DIRECT MORTGAGE',
    'NATIONS DIRECT MORTGAGE LLC': 'NATIONS DIRECT MORTGAGE',
    'AMERICAN INTERNET MORTGAGE, INC. DBA AIMLOAN.COM': 'AMERICAN INTERNET MORTGAGE',
    'CAM XIV TRUST': 'CAM TRUST',
    'CAM MORT TRUST': 'CAM TRUST',
    'ROUNDPOINT MORTGAGE SERVICING CORPORATION': 'ROUNDPOINT MORTGAGE SERVICING',
    'RESIDENTIAL CREDIT OPPORTUNITY V': 'RESIDENTIAL CREDIT SOLUTIONS',
    'RESIDENTIAL CREDIT SOLUTION': 'RESIDENTIAL CREDIT SOLUTIONS',
    'RESIDENTIAL CREDIT SOLUTIONS, INC.': 'RESIDENTIAL CREDIT SOLUTIONS',
    'LC EQUITY GROUP INC': 'LC EQUITY GROUP',
    'CENLAR FSB,': 'CENLAR FSB',
    'PACIFIC UNION FINANCIAL, LLC': 'PACIFIC UNION FINANCIAL',
    'CAPITAL ONE, NA': 'CAPITAL ONE',
    'LEWIA H. TENNIS': 'LEWIA TENNIS',
    'CENTER STREET LENDING FUND IV SPE, LLC, A DELAWARE LIMITED LIABILITY COMPANY': 'CENTER STREET LENDING FUND',
    'CENTER STREET LENDING FUND V, LLC, A DELAWARE LIMITED LIABILITY COMPANY': 'CENTER STREET LENDING FUND',
    'PARKER FORECLOSURE SERVICES, LLC': 'PARKER FORECLOSURE SERVICES',
    'PEAK FORECLOSURE SERVICES, INC.': 'PARKER FORECLOSURE SERVICES',
    'LOGIX FEDERAL CREDIT UNION FKA LOCKHEED FEDERAL CREDIT UNION': 'LOGIX FEDERAL CREDIT UNION',
    'ALBERT R SILVA TRUST': 'ALBERT R. SILVA TRUST',
    'ANCHOR LOANS, LP': 'ANCHOR LOANS',
    'CENTER FSB,': 'CENTER FSB',
    'FLAGSTAR BANK FSB': 'FLAGSTAR',
    'FLAGSTAR BANK': 'FLAGSTAR',
    'MIDWEST LOAN SERVICES INC': 'MIDWEST LOAN SERVICES',
    'CHRISTIANA TRUST,': 'CHRISTIANA TRUST',
    'CENTER STREET LENDING FUND IV SPE, LLC, A DELAWARE LIMITED LIABILITY COMPANY': 'CENTER STREET LENDING FUND',
    'GASBARRO INVESTMENTS INC': 'GASBARRO INVESTMENT',
    'GASBARRO INVESTMENTS, INC.': 'GASBARRO INVESTMENT',
    'REAL TIME RESOLUTIONS, INC.': 'REAL TIME RESOLUTIONS',
    'REAL TIME RESOLUTIONS, INC': 'REAL TIME RESOLUTIONS',
    'FAY SERVICING LLC - ERIC MOORE': 'FAY SERVICING',
    'CITY NATIONAL BANK': 'CITY NATIONAL FINANCIAL',
    'YAKTE PROPERTIES LLC': 'YAKTE PROPERTIES',
    'ABS REO TRUST II': 'ABS REO TRUST',
    'ABS REO TRUST V, NA,': 'ABS REO TRUST',
    'PLM LOAN MANAGEMENT SERVICES, INC.': 'PLM LOAN MANAGEMENT SERVICES',
    'PLM LOAN MANAGEMENT SERVICES, INC': 'PLM LOAN MANAGEMENT SERVICES',
    'BROUGHAM FUND I TRUST': 'BROUGHAM FUND',
    'CHRISTIANA TRUST, A DIVISION OF WILMINGTON SAVINGS FUND SOCIETY, FSB, NOT IN ITS INDIVIDUAL CAPACITY': 'CHRISTIANA TRUST',
    'MR. COOPER LLC': 'MR. COOPER',
    'MR COOPER': 'MR. COOPER',
    'KEYSTONE REAL ESTATE LENDING FUND, L.P.': 'KEYSTONE REAL ESTATE LENDING FUND',
    'KEYSTONE REAL ESTATE LENDING FUND LP': 'KEYSTONE REAL ESTATE LENDING FUND',
    'NPI DEBT FUND I LP': 'NPI DEBT FUND',
    'NPI DEBT FUND I, LP': 'NPI DEBT FUND',
    'NPI DEBT FUND I LP': 'NPI DEBT FUND',
    'NPI  DEBT FUND I, LP': 'NPI DEBT FUND',
    'ANCHOR LOANS, LP': 'ANCHOR LOANS',
    'JOEL S. LEVINE TRUSTEE OF THE JOEL S. LEVINE REVOCABLE TRUST DATED 2/28/1991': 'JOEL S. LEVINE',
    'INTERNATIONAL CITY MORTGAGE, INC': 'INTERNATIONAL CITY MORTGAGE',
    'AMWEST FUNDING CORP': 'AMWEST FUNDING',
    'AMWEST FUNDING CORP,': 'AMWEST FUNDING',
    'AM WEST FUNDING CORP': 'AMWEST FUNDING',
    'ABLE INVESTMENT SERVICES, LLC': 'ABLE INVESTMENT SERVICES',
    'D&A INTERNATIONAL INVESTMENT GROUP, LLC': 'D&A INTERNATIONAL INVESTMENT GROUP',
    'D&A INTERNATIONAL INVESTMENT GROUP LLC.': 'D&A INTERNATIONAL INVESTMENT GROUP',
    'EASY REALTY AND LOANS INC, A CALIFORNIA CORPORATION, BENEFICIARY': 'EASY REALTY AND LOANS',
    'FIRST AMERICAN TRUSTEE SERVICING SOLUTIONS, FIRST ENTERTAINMENT': 'FIRST AMERICAN TRUSTEE SERVICING SOLUTIONS',
    'FIRST AMERICAN TRUSTEE SERVICING SOLUTIONS - NATION DIRECT': 'FIRST AMERICAN TRUSTEE SERVICING SOLUTIONS',
    'FIRST AMERICAN TRUSTEE SERVICING SOLUTIONS - JEFFERSON': 'FIRST AMERICAN TRUSTEE SERVICING SOLUTIONS',
    'FIRST AMERICAN TRUSTEE SERVICING SOLUTIONS - LOGIX': 'FIRST AMERICAN TRUSTEE SERVICING SOLUTIONS',
    'NATIONSTAR MORTGAGE, DBA MR. COOPER': 'NATIONSTAR MORTGAGE',
    'PS FUNDING, INC': 'PS FUNDING',
    'PS FUNDING, INC.': 'PS FUNDING',
    'VAL-CHRIS INVESTMENTS INC.': 'VAL-CHRIS INVESTMENT',
    'VAL-CHRIS INVESTMENTS, INC.': 'VAL-CHRIS INVESTMENT',
    'VAL-CHRIS INVESTMENTS INC': 'VAL-CHRIS INVESTMENT',
    'VAL-CHRIS INVESTMENTS': 'VAL-CHRIS INVESTMENT',
    'REDWOOD BPL HOLDINGS, INC.': 'REDWOOD BPL HOLDINGS',
    'REDWOOD BPL HOLDINGS INC.': 'REDWOOD BPL HOLDINGS',
    'HOME POINT FINANCIAL CORPORATION': 'HOME POINT FINANCIAL',
    '2005 RESIDENTIAL TRUST 3-1': '2005 RESIDENTIAL TRUST',
    '2005 RESIDENTIAL TRUST 3-2': '2005 RESIDENTIAL TRUST',
    '5 AIF SYCAMORE2, LLC': 'AIF',
    '5 AIF SYCAMORE 2, LLC': 'AIF',
    '5AIF JUNIPER 2, LLC': 'AIF',
    '5 AIF SYCAMORE2, LLC': 'AIF',
    '5 AIF BAOBAB, LLC': 'AIF',
    '5AIF NUTMEG, LLC': 'AIF',
    '5AIF SYCAMORE 2, LLC': 'AIF',
    'FANNIE MAE - CARE OF PEMCO-LIMITED': 'FANNIE MAE',
    'PLANET HOME LENDING, LLC': 'PLANET HOME LENDING',
    'PANET HOME LENDING, LLC': 'PLANET HOME LENDING',
    'STATEBRIDGE COMPANY, LLC': 'STATEBRIDGE COMPANY',
    'STATEBRIDGE COMPANY LLC': 'STATEBRIDGE COMPANY',
    'TROJAN CAPITAL INVESTMENTS, LLC': 'TROJAN CAPITAL INVESTMENTS',
    'TROJAN CAPITAL INVESTMENTS, LLC': 'TROJAN CAPITAL INVESTMENTS', 
    'TROJAN CAPITAL INVESTMENT, LLC': 'TROJAN CAPITAL INVESTMENTS',
    'REAL TIME RESOLUTIONS, INC.': 'REAL TIME RESOLUTIONS',
    'REAL TIME RESOLUTIONS, INC.': 'REAL TIME RESOLUTIONS',
    'REAL TIME RESOLUTIONS, INC': 'REAL TIME RESOLUTIONS',
    'WEST COAST CAPITAL, LLC': 'WEST COAST CAPITAL',
    'WEST COAST CAPITAL, LLC': 'WEST COAST CAPITAL',
    'WEST COAST CAPITAL GROUP, INC.': 'WEST COAST CAPITAL',
    'MAXVI 1, LLC,': 'MAXVI 1 LLC',
    'KEYSTONE REAL ESTATE LENDING FUND LP': 'KEYSTONE REAL ESTATE LENDING FUND, L.P.',
    'BANK OF AMERICA NA': 'BANK OF AMERICA',
    'BANK OF AMERICA - PRE': 'BANK OF AMERICA',
    'WESCOM CENTRAL CREDIT UNION': 'WESCOM CREDIT UNION',
    'WANG YANG ENTERPRISES LLC': 'WANG YANG ENTERPRISES',
    'WANG YANG ENTERPRISES, LLC': 'WANG YANG ENTERPRISES',
    'BANC OF CALIFORNIA NATIONAL ASSOTIATION': 'BANC OF CALIFORNIA NATIONAL ASSOCIATION',
    'VILLAGE CAPITAL &  INVESTMENT, LLC': 'VILLAGE CAPITAL & INVESTMENT',
    'PARTNERS FOR PAYMENT RELIEF DE II, LLV': 'PARTNERS FOR PAYMENT RELIEF',
    'PARTNERS FOR PAYMENT RELIEF DE III, LLC': 'PARTNERS FOR PAYMENT RELIEF',
    '5AIF SYCAMORE 2, LLC': 'AIF',
    '5 AIF SYCAMORE 2, LLC': 'AIF',
    '5AIF JUNIPER 2, LLC': 'AIF',
    '5 AIF SYCAMORE2, LLC': 'AIF',
    '5AIF NUTMEG, LLC': 'AIF',
    '5AIF SYCAMORE 2, LLC': 'AIF',
    'DAKOTA ASSET SERVICES, LLC': 'DAKOTA ASSET SERVICES',
    'THE RAMA FUND LLC': 'THE RAMA FUND',
    'THE RAMA FUND, LLC': 'THE RAMA FUND',
    'GREGORY FUNDING LLC': 'GREGORY FUNDING',
    'ONSLOW BAY FINANCIAL LLC CO SELECT PORTFOLIO SERVICING': 'ONSLOW BAY FINANCIAL',
    'FRANKLIN CREDIT MANAGEMENT CORP.': 'FRANKLIN CREDIT MANAGEMENT CORPORATION',
    'FRANKLIN CREDIT MANAGEMENT CORP': 'FRANKLIN CREDIT MANAGEMENT CORPORATION',
    'TGP OPPORTUNITY FUND I LLC': 'TGP OPPORTUNITY FUND',
    'TGP OPPORTUNITY FUND I, LLC': 'TGP OPPORTUNITY FUND',
    'PHH MORGAGE CORPORATION': 'PHH MORTGAGE CORPORATION',
    'PHH MORTGAGE CORP': 'PHH MORTGAGE CORPORATION',
    'THE BANK OF NEW YORK MELLON, SUCCESSOR TO THE BANK OF NEW YORK, NOT IN ITS INDIVIDUAL CAPACITY BUT S': 'THE BANK OF NEW YORK MELLON',
    'SALAGUBANG, JOSE': 'JOSE SALAGUBANG',
    'SALAGUBANG,JOSE': 'JOSE SALAGUBANG',
    'SALAGUBANG JOSE': 'JOSE SALAGUBANG',
    'SALAGUBANG, JOSE': 'JOSE SALAGUBANG',
    'THE EMPLOYEES RETIREMENT AND BENEFIT PLAN AND TRUST': 'EMPLOYEES RETIREMENT & BENEFIT P&T',
    'CTF ASSET MANGEMENT, LLC': 'CTF ASSET MANAGEMENT',
    'CTF ASSET MANAGEMENT LLC': 'CTF ASSET MANAGEMENT',
    'SN SERVICING CO': 'SN SERVICING',
    'SN SERVICING CORP': 'SN SERVICING',
    'SN SERVICING CORPORATION': 'SN SERVICING',
    'MIDWEST LOAN SERVICES INC': 'MIDWEST LOAN SERVICES',
    'LC EQUITY GROUP INC': 'LC EQUITY GROUP',
    'POPA FEDERAL CREDITI UNION': 'POPA FEDERAL CREDIT UNION',
    'AJAX E MASTER TRUST I, A DELAWARE TRUST, WILMINGTON SAVINGS FUND SOCIETY, FSB, TRUSTEE': 'AJX MORTGAGE TRUST',
    'AJX MORTGAGE TRUST II, A DELAWARE TRUST, WILMINGTON SAVINGS FUND SOCIETY, FSB, TRUSTEE': 'AJX MORTGAGE TRUST',
    'AJX MORTGAGE TRUST I, A DELAWARE TRUST, WILMINGTON SAVINGS FUND SOCIETY, FSB, TRUSTEE': 'AJX MORTGAGE TRUST',
    'AJAX MORTGAGE LOAN TRUST 2018-G, MORTGAGE-BACKED SECURITIES, SERIES 2018-G, BY U.S. BANK NATIONAL AS': 'AJX MORTGAGE TRUST',
    'AJAX MORTGAGE LOAN TRUST 2018-D, MORTGAGE-BACKED SECURITIES, SERIES 2018-D, BY U.S. BANK NATIONAL AS': 'AJX MORTGAGE TRUST',
    'AJAX MORTGAGE LOAN TRUST 2019-A, MORTGAGE-BACKED SECURITIES, SERIES 2019-A, BY U.S. BANK NATIONAL AS': 'AJX MORTGAGE TRUST',
    'AJAX MORTGAGE LOAN TRUST 2019-E, MORTGAGE-BACKED SECURITIES, SERIES 2019-E, BY U.S. BANK NATIONAL AS': 'AJX MORTGAGE TRUST',
    'AJAX MORTGAGE LOAN TRUST 2018-G, MORTGAGE-BACKED SECURITIES, SERIES 2018-G, BY U.S. BANK NATIONAL AS': 'AJX MORTGAGE TRUST',
    'AJX MORTGAGE TRUST II, A DELAWARE TRUST, WILMINGTON SAVINGS FUND SOCIETY, FSB, TRUSTEE': 'AJX MORTGAGE TRUST',
    'AJAX E MASTER TRUST I, A DELAWARE TRUST, WILMINGTON SAVINGS FUND SOCIETY, FSB, TRUSTEE': 'AJX MORTGAGE TRUST',
    'AJX MORTGAGE TRUST I, A DELAWARE TRUST, WILMINGTON SAVINGS FUND SOCIETY, FSB, TRUSTEE': 'AJX MORTGAGE TRUST',
    'INVESTMENT MANAGEMENT COMPANY, LLC': 'INVESTMENT MANAGEMENT COMPANY',
    'DMI - HSBC': 'HSBC',
    'INTERNATIONAL CITY MORTGAGE, INC': 'INTERNATIONAL CITY MORTGAGE',
    'DEUTSCHE BANK NATIONAL TRUST CO. AS TR FOR SOUNDVIEW 2006 OPT-3': 'DEUTSCHE BANK',
    'GASBARRO INVESTMENTS, INC.': 'GASBARRO INVESTMENT',
    'GASBARRO INVESTMENTS INC': 'GASBARRO INVESTMENT',
    'GASBARRO INVESTMENTS, INC.': 'GASBARRO INVESTMENT',
    'D&A INTERNATIONAL INVESTMENT GROUP LLC.': 'D&A INTERNATIONAL INVESTMENT GROUP',
    'D&A INTERNATIONAL INVESTMENT GROUP LLC.': 'D&A INTERNATIONAL INVESTMENT GROUP',
    'D&A INTERNATIONAL INVESTMENT GROUP, LLC': 'D&A INTERNATIONAL INVESTMENT GROUP',
    'ZELAYA, CARLOS E': 'CARLOS E ZELAYA',
    'ZELAYA,CARLOS E': 'CARLOS E ZELAYA',
    'AKSELROD BYPASS TRUST': 'AKSELROD TRUST',
    'AKSELROD FAMILY TRUST': 'AKSELROD TRUST',
    'M&T BANK - JEFFERSON OFFICE': 'M&T BANK',
    'SELENE FINANCE - JEFFERSON OFFICE': 'SELENE FINANCE',
    'SN SERVICING CO': 'SN SERVICING',
    'SN SERVICING CORP': 'SN SERVICING',
    'SN SERVICING CORPORATION': 'SN SERVICING',
    'AZITA FRIEDA DELRAHIM': 'AZITA DELRAHIM',
    'PNC BANK, NATIONAL ASSOCIATION,': 'PNC',
    'HUDSON HOMES MANAGEMENT LLC': 'HUDSON HOMES MANAGEMENT',
    'THE BANK OF NEW YORK MELLON, DITECH FINANCIAL LLC': 'THE BANK OF NEW YORK MELLON',
    'THE BANK OF NEW YORK MELLON FKA THE BANK OF NEW YORK': 'THE BANK OF NEW YORK MELLON',
    'FIDUCIARY TRUST INTERNATIONAL OF CA': 'FIDUCIARY TRUST INTERNATIONAL OF CALIFORNIA',
    'OLIVER W TAYLOR': 'OLIVER W & NORMA J TAYLOR',
    'WITKIN & EISINGER LLC': 'WITKIN & ASSOCIATES',
    'WITKIN & ASSOCIATES LLC': 'WITKIN & ASSOCIATES',
    'WITKIN & ASSOCIATES, LLC': 'WITKIN & ASSOCIATES',
    'CTF ASSET MANAGEMENT LLC': 'CTF ASSET MANGEMENT',
    'CTF ASSET MANGEMENT, LLC': 'CTF ASSET MANGEMENT',
    '5 AIF BAOBAB, LLC': 'AIF',
    'BOURNS EMPLOYEES FEDERAL CREDIT CREDIT': 'BOURNS EMPLOYEES FEDERAL CREDIT UNION',
    'MORTGAGE MANAGEMENT CONSULTANTS, INC': 'MORTGAGE MANAGEMENT CONSULTANTS',
    'FINGERGUT LEON': 'LEON FINGERGUT',
    'FINGERGUT,LEON': 'LEON FINGERGUT',
    'TGP OPPORTUNITY FUND I LLC': 'TGP OPPORTUNITY FUND I, LLC',
    'LEWIA H. TENNIS': 'LEWIA TENNIS',  
    'TOORAK CAPITAL PARTNERS, LLC': 'TOORAK CAPITAL PARTNERS',
    'TOORAK CAPITAL PARTNERS LLC': 'TOORAK CAPITAL PARTNERS',
    'FINANCE OF AMERICA REVERSE': 'FINANCE OF AMERICA',
    'FINANCE OF AMERICA COMMERCIAL': 'FINANCE OF AMERICA',
    'TOWD POINT MORTGAGE TRUST 2018-1, U.S. BANK NATIONAL ASSOCIATION, AS INDENTURE TRUSTEE': 'U.S. BANK',
    'PS FUNDING, INC.': 'PS FUNDING',
    'PS FUNDING, INC': 'PS FUNDING',
    'WELLS FARGO BANK NA TR / NATIONSTAR MORTGAGE, LLC.': 'WELLS FARGO BANK',
    'WILMINGTON TRUST, NATIONAL ASSOCIATION, NOT IN ITS INDIVIDUAL CAPACITY BUT SOLELY MFRA TRUST 2015-1': 'WILMINGTON TRUST',
    'WILMINGTON SAVINGS FUND SOCIETY, FSB, NOT IN ITS INDIVIDUAL CAPACITY BUT SOLELY AS OWNER TRUSTEE': 'WILMINGTON TRUST',
    'WILMINGTON SAVINGS FUND SOCIETY, FSB, AS OWNER TRUSTEE OF THE RESIDENTIAL CREDIT OPPORTUNITIES TRUS': 'WILMINGTON TRUST',
    'WILMINGTON SAVINGS FUND SOCIETY, FSB, AS OWNER TRUSTEE OF THE RESIDENTIAL CREDIT OPPORTUNITIES TRUS': 'WILMINGTON TRUST',
    'LIMA ONE CAPITAL LLC': 'LIMA ONE CAPITAL',
    'FREDDIE MAC HOME LOAN CORPORATION': 'FREDDIE MAC',
    'EMPLOYEES RETIREMENT & BENEFIT P&T': 'THE EMPLOYEES RETIREMENT AND BENEFIT PLAN AND TRUST',
    'ABLE INVESTMENT SERVICES, LLC': 'ABLE INVESTMENT SERVICES',
    'HUDSON HOMES MANAGEMENT LLC': 'HUDSON HOMES MANAGEMENT',
    'QUIJADA ERNESTO': 'ERNESTO QUIJADA',
    'QUIJADA,ERNESTO': 'ERNESTO QUIJADA',
    'AFGHAN AMERICAN TRADING COMPANY INC.': 'AFGHAN AMERICAN TRADING COMPANY',
    'AFGHAN AMERICAN TRADING CO.': 'AFGHAN AMERICAN TRADING COMPANY',
    'AFGHAN AMERICAN TRADING COMPANY INC.': 'AFGHAN AMERICAN TRADING COMPANY',
    'CITIGROUP MORTGAGE LOAN TRUST INC': 'CITIBANK',
    'STRUNZO DEVELOPMENT CORPORATION OF CALIFORNIA': 'STRUNZO DEVELOPMENT CORPORATION',
    'MAXVI 1, LLC,': 'MAXVI 1 LLC',
    'WILMINGTON SAVINGS FUND SOCIETY, FSB, NOT IN ITS INDIVIDUAL CAPACITY BUT SOLELY AS OWNER TRUSTEE': 'WILMINGTON TRUST',
    'WILMINGTON TRUST, NATIONAL ASSOCIATION, NOT IN ITS INDIVIDUAL CAPACITY BUT SOLELY MFRA TRUST 2015-1': 'WILMINGTON TRUST',
    'NPI DEBT FUND I, LP': 'NPI DEBT FUND',
    'NPI DEBT FUND I LP': 'NPI DEBT FUND',
    'NPI  DEBT FUND I, LP': 'NPI DEBT FUND',
    'TOORAK CAPITAL PARTNERS LLC': 'TOORAK CAPITAL PARTNERS',
    'TOORAK CAPITAL PARTNERS, LLC': 'TOORAK CAPITAL PARTNERS',
    'TOORAK CAPITAL PARTNERS LLC': 'TOORAK CAPITAL PARTNERS',
    'TOORAK CAPITAL PARTNERS': 'TOORAK CAPITAL PARTNERS',
    'MUFG UNION BANK, N.A.': 'MUFG UNION BANK',
    'SKYLARK CAPITAL MANAGEMENT LLC': 'SKYLARK CAPITAL MANAGEMENT',
    'AHP SERVICING LLC': 'AHP SERVICING'}

LA_foreclosures_2015_to_2022['lender'].replace(lender_mapping, inplace=True)


In [57]:
# Replace old lender names with new names in place
LA_foreclosures_2015_to_2022['lender'].replace(lender_mapping, inplace=True)

# Display the updated DataFrame
print(LA_foreclosures_2015_to_2022['lender'].unique())

['WELLS FARGO BANK' 'JP MORGAN CHASE' 'DEUTSCHE BANK' ...
 'WELLS FARGO AS TR OF OPTION ONE MORTGAGE LOAN TRUST SERIES 2007-FXD1, BY SLS-COMPUTERSHARE AS ATTY'
 'KAT INVESTMENT TRUST & DMT INVESTMENT TRUST'
 'ROYAL PACIFIC FUNDING CORP']


In [58]:
#I realize that there are a few coordinates nested in the 'property_address'. 
#I will remove them and constrain the address to just end at the Ave or street

# Split the address column by "\n" and keep only the first part
LA_foreclosures_2015_to_2022['property_address'] = LA_foreclosures_2015_to_2022['property_address'].str.split('\n').str[0]

# Display the DataFrame after modification
print(LA_foreclosures_2015_to_2022['property_address'])

311            346 E 116TH ST
331        19531 W COLLIER ST
332        1329 W LINWOOD AVE
333         7870 S BOEING AVE
334       6941 OWENSMOUTH AVE
                 ...         
29753    7753 N BOTHWELL ROAD
29754    10513 N SHOSHONE AVE
29755          7436 W 88TH PL
29756      20836 S DENKER AVE
29757      10339 N CAYUGA AVE
Name: property_address, Length: 28954, dtype: object


In [59]:
#changing zip to integer
LA_foreclosures_2015_to_2022['property_zip'] = LA_foreclosures_2015_to_2022['property_zip'].astype(int)

# Display the DataFrame to verify the changes
print(LA_foreclosures_2015_to_2022['property_zip'])


311      90061
331      91356
332      90017
333      90045
334      91303
         ...  
29753    91335
29754    91344
29755    90045
29756    90501
29757    91331
Name: property_zip, Length: 28954, dtype: int64


In [60]:
LA_foreclosures_2015_to_2022.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28954 entries, 311 to 29757
Data columns (total 15 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   apn                         28954 non-null  int64  
 1   registered_date             28954 non-null  object 
 2   property_type               28954 non-null  object 
 3   property_address            28954 non-null  object 
 4   property_city               28954 non-null  object 
 5   property_state              28954 non-null  object 
 6   property_zip                28954 non-null  int64  
 7   council_district            28954 non-null  int64  
 8   lender                      28954 non-null  object 
 9   lender_contact              28954 non-null  object 
 10  property_management         28954 non-null  object 
 11  longitude                   28954 non-null  float64
 12  latitude                    28954 non-null  float64
 13  foreclosure_year            28954 

In [61]:
#renaming the foreclosure_year column to join with homeless dataframe
LA_foreclosures_2015_to_2022.rename(columns={'foreclosure_year': 'year'}, inplace=True)


In [62]:
#removing leading or trailing whitespaces

LA_foreclosures_2015_to_2022 = LA_foreclosures_2015_to_2022.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Display the DataFrame after removing leading and trailing whitespaces
print(LA_foreclosures_2015_to_2022)

              apn         registered_date  property_type  \
311    6083008010              01/12/2015   Multi-Family   
331    2164023003              22/01/2015  Single Family   
332    5143012013              22/04/2015  Single Family   
333    4106029005              15/07/2015  Single Family   
334    2138005033              24/08/2015  Single Family   
...           ...                     ...            ...   
29753  2116008035  12/21/2022 12:00:00 AM  Single Family   
29754  2733011004  09/27/2022 12:00:00 AM  Single Family   
29755  4119020006  06/07/2022 12:00:00 AM  Single Family   
29756  7351025018  08/05/2022 12:00:00 AM  Single Family   
29757  2620020002  11/03/2022 12:00:00 AM  Single Family   

           property_address property_city property_state  property_zip  \
311          346 E 116TH ST   LOS ANGELES             CA         90061   
331      19531 W COLLIER ST   LOS ANGELES             CA         91356   
332      1329 W LINWOOD AVE   LOS ANGELES             CA 

## checking the unique values in the columns to see what needs to be cleaned 

In [63]:
#Finding the unique values of property_city

unique_property_cities = LA_foreclosures_2015_to_2022['property_city'].unique()
number_unique_property_cities = LA_foreclosures_2015_to_2022['property_city'].value_counts()

# Display the unique values
print("Unique values in the 'property_city' column:")
print(unique_property_cities)

# Display the number of unique values
print("Number of unique values in the 'property_city' column:")
print(number_unique_property_cities)

Unique values in the 'property_city' column:
['LOS ANGELES' 'CANOGA PARK' 'WESTCHESTER' 'HOLLYWOOD' 'VENICE' 'SYLMAR'
 'SAN PEDRO' 'VAN NUYS' 'TARZANA' 'MARINA DEL REY' 'NORTH HOLLYWOOD'
 'HARBOR CITY' 'PLAYA DEL REY' 'WOODLAND HILLS' 'WINNETKA' 'PANORAMA CITY'
 'WILMINGTON' 'BAD APN' 'ENCINO' 'PACOIMA' 'NORTH HILLS' 'TORRANCE'
 'NORTHRIDGE' 'SHERMAN OAKS' '# 1 LOS ANGELES' 'L.A.' 'TUJUNGA'
 'STUDIO CITY' 'CHATSWORTH' 'LOMITA' 'GARDENA' 'VALLEY VILLAGE' 'RESEDA'
 'BURBANK' 'LAKE VIEW TERRACE' 'MISSION HILLS' 'SUN VALLEY' 'LA'
 'LOS SNGELES' 'OLD APN' 'SAN FERNANDO' 'RANCHO LA TUNA CANYON'
 'BEVERLY HILLS' 'SUNLAND' 'PACIFIC PALISADES' 'SEPULVEDA' 'ARLETA'
 'SANTA MONICA']
Number of unique values in the 'property_city' column:
property_city
LOS ANGELES              26983
VAN NUYS                   600
SAN PEDRO                  438
HOLLYWOOD                  284
NORTH HOLLYWOOD            102
VENICE                      64
SYLMAR                      48
TARZANA                     42
WO

In [64]:
#Show rows the info on the rows with 'bad APN' and 'old APN'

# Filter rows where 'property_city' value is 'BAD APN'
bad_apn_rows = LA_foreclosures_2015_to_2022[LA_foreclosures_2015_to_2022['property_city'] == 'BAD APN']

# Display the information on the filtered rows
print("Information on rows where 'property_city' value is 'BAD APN':")
print(bad_apn_rows)

Information on rows where 'property_city' value is 'BAD APN':
              apn         registered_date  property_type   property_address  \
1282   6036021011              03/11/2015  Single Family   8942 S WILTON PL   
4458   2651008079              13/07/2015  Single Family  9229 N TOBIAS AVE   
14669  6036021011              01/08/2016  Single Family   8942 S WILTON PL   
16539  5109008054              01/02/2017   Multi-Family      237 E 51ST ST   
16783  5109008054              01/01/2018   Multi-Family      237 E 51ST ST   
19718  5109008054              07/19/2019   Multi-Family      237 E 51ST ST   
20058  5109008054              01/01/2019   Multi-Family      237 E 51ST ST   
23870  5109008054              01/20/2020   Multi-Family      237 E 51ST ST   
26208  5109008054  01/04/2021 12:00:00 AM   Multi-Family      237 E 51ST ST   

      property_city property_state  property_zip  council_district  \
1282        BAD APN             CA         90047                 8   
4458   

In [65]:
# Filter rows where 'property_city' value is 'OLD APN'
old_apn_rows = LA_foreclosures_2015_to_2022[LA_foreclosures_2015_to_2022['property_city'] == 'OLD APN']

# Display the information on the filtered rows
print("Information on rows where 'property_city' value is 'OLD APN':")
print(old_apn_rows)

Information on rows where 'property_city' value is 'OLD APN':
              apn         registered_date  property_type  \
7371   5081019014              09/04/2015  Single Family   
9615   5081019014              01/04/2016  Single Family   
15038  5081019014              01/03/2017  Single Family   
16920  5081019014              01/01/2018  Single Family   
21422  5081019014              08/07/2019  Single Family   
22843  5081019014              01/16/2020  Single Family   
28495  5081019014  04/14/2022 12:00:00 AM  Single Family   

           property_address property_city property_state  property_zip  \
7371   1101 S ARLINGTON AVE       OLD APN             CA         99999   
9615   1101 S ARLINGTON AVE       OLD APN             CA         99999   
15038  1101 S ARLINGTON AVE       OLD APN             CA         99999   
16920  1101 S ARLINGTON AVE       OLD APN             CA         99999   
21422  1101 S ARLINGTON AVE       OLD APN             CA         99999   
22843  1101 S

In [66]:
#Renaming a few  cities

# Define a dictionary mapping old values to new values
city_renaming = {'L.A.': 'Los Angeles',
                'LA': 'Los Angeles',
                '# 1 LOS ANGELES': 'Los Angeles',
                'LOS SNGELES': 'Los Angeles'}

# Replace values in the 'property_city' column
LA_foreclosures_2015_to_2022['property_city'] = LA_foreclosures_2015_to_2022['property_city'].replace(city_renaming)

# Display the DataFrame after modification
print(LA_foreclosures_2015_to_2022)


              apn         registered_date  property_type  \
311    6083008010              01/12/2015   Multi-Family   
331    2164023003              22/01/2015  Single Family   
332    5143012013              22/04/2015  Single Family   
333    4106029005              15/07/2015  Single Family   
334    2138005033              24/08/2015  Single Family   
...           ...                     ...            ...   
29753  2116008035  12/21/2022 12:00:00 AM  Single Family   
29754  2733011004  09/27/2022 12:00:00 AM  Single Family   
29755  4119020006  06/07/2022 12:00:00 AM  Single Family   
29756  7351025018  08/05/2022 12:00:00 AM  Single Family   
29757  2620020002  11/03/2022 12:00:00 AM  Single Family   

           property_address property_city property_state  property_zip  \
311          346 E 116TH ST   LOS ANGELES             CA         90061   
331      19531 W COLLIER ST   LOS ANGELES             CA         91356   
332      1329 W LINWOOD AVE   LOS ANGELES             CA 

In [67]:
#checking if everything worked

print(LA_foreclosures_2015_to_2022['property_city'].value_counts())

property_city
LOS ANGELES              26983
VAN NUYS                   600
SAN PEDRO                  438
HOLLYWOOD                  284
NORTH HOLLYWOOD            102
VENICE                      64
SYLMAR                      48
TARZANA                     42
WOODLAND HILLS              41
WILMINGTON                  41
CANOGA PARK                 35
ENCINO                      24
PACOIMA                     23
RESEDA                      22
CHATSWORTH                  18
TORRANCE                    17
Los Angeles                 13
NORTHRIDGE                  12
WESTCHESTER                 12
HARBOR CITY                 11
SHERMAN OAKS                 9
BURBANK                      9
STUDIO CITY                  9
BAD APN                      9
PANORAMA CITY                8
PLAYA DEL REY                7
WINNETKA                     7
MARINA DEL REY               7
SUN VALLEY                   7
OLD APN                      7
BEVERLY HILLS                6
VALLEY VILLAGE           

In [68]:
#making things Title case

LA_foreclosures_2015_to_2022['property_city'] = LA_foreclosures_2015_to_2022['property_city'].str.title()

# Display the DataFrame after modification
print(LA_foreclosures_2015_to_2022['property_city'])



311      Los Angeles
331      Los Angeles
332      Los Angeles
333      Los Angeles
334      Canoga Park
            ...     
29753    Los Angeles
29754    Los Angeles
29755    Los Angeles
29756    Los Angeles
29757    Los Angeles
Name: property_city, Length: 28954, dtype: object


In [69]:
LA_foreclosures_2015_to_2022['property_zip'].info()

<class 'pandas.core.series.Series'>
Index: 28954 entries, 311 to 29757
Series name: property_zip
Non-Null Count  Dtype
--------------  -----
28954 non-null  int64
dtypes: int64(1)
memory usage: 452.4 KB


In [70]:
#Weird fix but looking at Tableau, I noticed there were a few coordinates that were pinned somewhere in the midwest. 
#so I want to see which zips are labeled 99999 as shown in the map

# Count the number of rows where the zip code is 99999
num_rows_with_99999_zip = (LA_foreclosures_2015_to_2022['property_zip'] == 99999).sum()

# Print the number of rows with zip code 99999
print("Number of rows with zip code 99999:", num_rows_with_99999_zip)



Number of rows with zip code 99999: 39


In [71]:
#drop the rows with 99999 as the zip code
LA_foreclosures_2015_to_2022.drop(LA_foreclosures_2015_to_2022[LA_foreclosures_2015_to_2022['property_zip'] == 99999].index, inplace=True)

In [72]:
#Finding the unique values of property_type

unique_property_types = LA_foreclosures_2015_to_2022['property_type'].unique()

# Display the unique values
print("Unique values in the 'property_type' column:")
print(unique_property_types)

Unique values in the 'property_type' column:
['Multi-Family' 'Single Family' 'Non-Residential' 'Vacant Residential']


In [73]:
#Dropping bad APN rows

LA_foreclosures_2015_to_2022.drop(LA_foreclosures_2015_to_2022[LA_foreclosures_2015_to_2022['property_city'] == 'BAD APN'].index, inplace=True)

In [74]:
#dropping old apn rows

LA_foreclosures_2015_to_2022.drop(LA_foreclosures_2015_to_2022[LA_foreclosures_2015_to_2022['property_city'] == 'OLD APN'].index, inplace=True)

In [75]:
LA_foreclosures_2015_to_2022.to_csv('LA_foreclosures_2015_to_2022.csv', index=False)

# Display the download link
FileLink('LA_foreclosures_2015_to_2022.csv')

/Users/jilliangraham/Documents/GitHub/Jill_quests_IH/AirB&E/LA_foreclosures_2015_to_2022.csv

# Finding some descriptive stats

## Finding unique values

In [76]:
LA_foreclosures_2015_to_2022['year'].unique()

array([2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022])

In [77]:
#finding the most common foreclosed property type

property_type_counts = LA_foreclosures_2015_to_2022['property_type'].value_counts()

print("Property type value counts:")
print(property_type_counts)

most_common_property_type = property_type_counts.idxmax()

print("\nThe most common foreclosed property type is:", most_common_property_type)



Property type value counts:
property_type
Single Family         24275
Multi-Family           4292
Non-Residential         222
Vacant Residential      126
Name: count, dtype: int64

The most common foreclosed property type is: Single Family


In [78]:
#finding the most common council district

most_common_district = LA_foreclosures_2015_to_2022['council_district'].value_counts().idxmax()

# Visualizing the data

In [79]:
LA_foreclosures_2015_to_2022

,apn,registered_date,property_type,property_address,property_city,property_state,property_zip,council_district,lender,lender_contact,property_management,longitude,latitude,year,distance_to_la_city_center
311,6083008010,01/12/2015,Multi-Family,346 E 116TH ST,Los Angeles,CA,90061,15,WELLS FARGO BANK,Jonathon Holt,LPS,-118.267830,33.929188,2015,13.858041
331,2164023003,22/01/2015,Single Family,19531 W COLLIER ST,Los Angeles,CA,91356,3,JP MORGAN CHASE,Amy Lott,MCS,-118.558790,34.166635,2015,31.677087
332,5143012013,22/04/2015,Single Family,1329 W LINWOOD AVE,Los Angeles,CA,90017,1,DEUTSCHE BANK,Leon Oransky,"US Best Repair Service, Inc",-118.268789,34.051050,2015,2.314952
333,4106029005,15/07/2015,Single Family,7870 S BOEING AVE,Los Angeles,CA,90045,11,BANK OF AMERICA,VPR Department,EDWARD ZUBIA,-118.387358,33.966885,2015,16.289111
334,2138005033,24/08/2015,Single Family,6941 OWENSMOUTH AVE,Canoga Park,CA,91303,3,SELECT PORTFOLIO SERVICING,Code Violations,MCS,-118.601545,34.196811,2015,36.654745
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29753,2116008035,12/21/2022 12:00:00 AM,Single Family,7753 N BOTHWELL ROAD,Los Angeles,CA,91335,3,LOANCARE,Eric MOORE,Safeguard,-118.561370,34.211720,2022,34.198443
29754,2733011004,09/27/2022 12:00:00 AM,Single Family,10513 N SHOSHONE AVE,Los Angeles,CA,91344,12,LOANCARE,Eric MOORE,Safeguard,-118.517800,34.261570,2022,34.323468
29755,4119020006,06/07/2022 12:00:00 AM,Single Family,7436 W 88TH PL,Los Angeles,CA,90045,11,CENLAR FSB,Eric Moore,Safeguard - Code Compliance,-118.425370,33.956490,2022,19.841929
29756,7351025018,08/05/2022 12:00:00 AM,Single Family,20836 S DENKER AVE,Los Angeles,CA,90501,15,PLANET HOME LENDING,Alyssa Guzman,Gary's Field Inspection Service,-118.303770,33.840820,2022,24.148683
